In [ ]:
from pandas_datareader import data as web
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import csv
from matplotlib import pyplot as plt
import random
import plotly.colors
from datetime import datetime
import time
import datetime as dt
import copy
import glob
import pickle

In [ ]:
def get_data(asset_name='SPY', directory='../../../data/yahoo_data', start_date='01-01-2005', stop_date='12-20-2019', force_yahoo=0, verbosity=0):
        
    # If the 'directory' doesn't exist, create it
    if os.path.isdir(directory) == False:
        os.mkdir(directory)

    pathname = directory + '/' + asset_name + '_' + start_date + '.csv'
    if os.path.isfile(pathname) == True and not force_yahoo:
        if verbosity >= 1:
            print('Loading from file: ', pathname)        
        df = pd.read_csv(pathname, index_col='Date')
    else:
        if verbosity >= 1:
            print('Downloading from Yahoo! - ', asset_name)
        df = web.DataReader(asset_name, data_source='yahoo', start=start_date, end=stop_date)
        df.to_csv(pathname)    

    # Copy the (date) index to a Date field and make a new index which enumerates
    # all the entries.
    df.insert(0, 'Date', df.index)
    df.index = np.arange(df.shape[0])
    df['Date']=pd.to_datetime(df['Date'], format='%Y/%m/%d')
    
    return df 

In [ ]:
# Synthetic stock generator

# We'll generate the Adj Close price as the primary (since this is the sequnce of labels we are training on)
# The relationship of the other features to the Adj Close will be as follows:
# Close = close_scale * Adj Close
# Open = previous day's Close
# Low = Open
# High = Close
# Volume = 0

# source_asset_name - name of asset to model from
# trend_list - list of tuples - (magnitude, days for a full cycle of this trend)

def make_synthetic_stock(source_asset_name='SPY', new_name='SYN', directory='../../../data/yahoo_data', start_date='01-01-2005', stop_date='12-20-2019', trend_list=[], adj_close_start=0, price_slope=0, close_scale=1.4):
    
    syn_df = get_data(asset_name=source_asset_name, directory=directory, start_date=start_date, stop_date=stop_date, force_yahoo=1)
        
    syn_length = len(syn_df)
    
    for i in range(syn_length):
        mag = adj_close_start + (i * price_slope)
        for trend in trend_list:
            mag += (trend[0] * np.sin(6.28 * i / trend[1]))

        syn_df['Adj Close'].loc[i] = mag
        syn_df['Close'].loc[i] = mag * close_scale
    
        # Today's open is yesterday's close
        if i > 0:
            syn_df['Open'].loc[i] = syn_df['Close'].loc[i-1]
        else:
            syn_df['Open'].loc[i] = syn_df['Close'].loc[i]
        
        syn_df['Low'].loc[i] = syn_df['Open'].loc[i]
        syn_df['High'].loc[i] = syn_df['Close'].loc[i]
    
    pathname = directory + '/' + new_name + '_' + start_date + '.csv'    
    
    syn_df.to_csv(pathname, index=False)    

    

In [ ]:
# Display a Plotly interactive candelstick chart

# path is directory to save image in
# display_plot - 1=display plot in Notebook

def plotly_candlestick(asset_df, asset_name, basefn = '', display_plot = 0, save_plot=1):

    df = asset_df
    
    trace1 = {
        'x': df.Date,
        'open': df.Open,
        'close': df.Close,
        'high': df.High,
        'low': df.Low,
        'type': 'candlestick',
        'name': asset_name,
        'showlegend': True
    }

    data = [trace1]
        
    layout = go.Layout({
        'title': {
            'text': 'Asset: ' + asset_name,
            'font': {
                'size': 20
            }
        }
    })

    fig = go.Figure(trace1, layout)
    
    if display_plot:        
        fig.show()
    
    if save_plot:
        fig.write_html(basefn + 'asset_candelstick.html')



In [ ]:
def add_features_to_df(df):
    # Add a feature called Prev Adj CLose
    prev_adj_close = df['Adj Close'].shift(1).copy()
    df['Prev Adj Close'] = prev_adj_close

    # Take care of special case of very first previous Adj close - just make is the same as the Adj Close
    df['Prev Adj Close'][0] = df['Prev Adj Close'][1]
    
    # Volume isn't used
    df = df.drop(columns=['Volume'])
    
    return df
    

In [ ]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

print("Cuda available:",torch.cuda.is_available())

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    


In [ ]:
# This will normalise each column of a dataframe to +/- 0.5

def normalise_df(df):
           
    r = df.copy()
    
    # Get the min and max of all values in dataframe except 'Date' field
    min_value = df.drop('Date',1).min().min()    
    max_value = df.drop('Date',1).max().max()

#    print(min_value)
#    print(max_value)
    
#    r['Open'] = (r['Open'] - min_value) / (max_value - min_value) - 0.5
#    r['Low'] = (r['Low'] - min_value) / (max_value - min_value) - 0.5
#    r['High'] = (r['High'] - min_value) / (max_value - min_value) - 0.5
#    r['Close'] = (r['Close'] - min_value) / (max_value - min_value) - 0.5
#    r['Adj Close'] = (r['Adj Close'] - min_value) / (max_value - min_value) - 0.5
#    r['Prev Adj Close'] = (r['Prev Adj Close'] - min_value) / (max_value - min_value) - 0.5    


    r.loc[:, r.columns != 'Date'] = (((r.loc[:, r.columns != 'Date'] - min_value) / (max_value - min_value)) - 0.5)
#    print(r.describe())
    
#    display(r)
    scale = max_value - min_value
    offset = min_value
#    To un-normalise:
    
#    original_value = ((norm_price + 0.5) * scale) + offset
#    scale = max_value - min_value
#    offset = min_value

#    print("values from df normalisation:")
#    print("Scale:", scale)
#    print("Offset:", offset)


#    display(r.head())
#    display(df.head())
    return r, scale, offset

In [ ]:
def denormalise_value(norm_value, scale, offset):
    return (((norm_value + 0.5) * scale) + offset)

In [ ]:
def unnormalise_df(df, scale, offset):
    
    result = df.copy()
    
    df = ((df + 0.5) * scale) + offset
    
    return(df)

In [ ]:
# This version is for sequence to value learning - the label is a single value - tomorrow's Adj Close

# Given one of the asset train, valid or test dataframes, this will make a new three dimensional list which has
# dimensions (num_samples, window_size - tw, 6) which represents the effect of sliding a window of width window_size (tw)
# down the dataframe from start to end and collecting the input features into a length 6 vector.  This will be our
# input dataset to the LSTM.

# In addition, also generate a list of (training) labels which is "tomorrow's" adjusted closing price - the thing 
# we are trying to predict. There is a single training label for each tw x 6 set of inputs.


def create_input_sequences_out_val_from_df(input_data_df, tw):
    in_seq = []
    out_seq= []
    L = len(input_data_df)

    for i in range(L-tw):
        seq = []
        for j in range(tw):
            features = [input_data_df['Open'].loc[i+j], input_data_df['Low'].loc[i+j], input_data_df['High'].loc[i+j], 
                        input_data_df['Close'].loc[i+j], 
                         input_data_df['Prev Adj Close'].loc[i+j], input_data_df['Adj Close'].loc[i+j]]
            seq.append(features)

        train_label = input_data_df['Adj Close'].loc[i+tw]
        out_seq.append(train_label)
        in_seq.append(seq)
    return in_seq, out_seq


In [ ]:
# This version is for sequence to sequence learning - the label is a sequence up to and including tomorrows Adj Close

def create_input_sequences_from_df(input_data_df, tw):
    in_seq = []
    out_seq= []
    L = len(input_data_df)

    for i in range(L-tw):
        window_input_seq = []
        window_output_seq = []
        for j in range(tw):
            features = [input_data_df['Open'].loc[i+j], input_data_df['Low'].loc[i+j], input_data_df['High'].loc[i+j], 
                        input_data_df['Close'].loc[i+j], 
                         input_data_df['Prev Adj Close'].loc[i+j], input_data_df['Adj Close'].loc[i+j]]
            window_input_seq.append(features)
            
            window_output_seq.append(input_data_df['Adj Close'].loc[i+j+1])

        out_seq.append(window_output_seq)
        in_seq.append(window_input_seq)
        
#        print("seq:", i)
#        print("input:\n",window_input_seq)
#        print("output:\n", window_output_seq)
    return in_seq, out_seq


In [ ]:
def get_continuous_color(colorscale, intermed):
    """
    Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
    color for any value in that range.

    Plotly doesn't make the colorscales directly accessible in a common format.
    Some are ready to use:
    
        colorscale = plotly.colors.PLOTLY_SCALES["Greens"]

    Others are just swatches that need to be constructed into a colorscale:

        viridis_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Viridis)
        colorscale = plotly.colors.make_colorscale(viridis_colors, scale=scale)

    :param colorscale: A plotly continuous colorscale defined with RGB string colors.
    :param intermed: value in the range [0, 1]
    :return: color in rgb string format
    :rtype: str
    """
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    if intermed <= 0 or len(colorscale) == 1:
        return colorscale[0][1]
    if intermed >= 1:
        return colorscale[-1][1]

    for cutoff, color in colorscale:
        if intermed >= cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    # noinspection PyUnboundLocalVariable
    return plotly.colors.find_intermediate_color(
        lowcolor=low_color, highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb")

In [ ]:
def plotly_gradients(model, min_grads, max_grads, ave_grads, basefn='', display_plot=0, save_plot=1):
    
    scale = plotly.colors.PLOTLY_SCALES["Bluered"]

    print("Number of gradients randomly sampled:", len(ave_grads))  
    
    
    # Assemble the names of the layers
    layers = []
    for n,p in model.named_parameters():
        if (p.requires_grad) and ("bias" not in n):
            layers.append(n)
    
    # Dong this the dumb way just now
    max_array = np.zeros(len(layers))
    min_array = np.ones(len(layers))
    mean_array = np.zeros(len(layers))
    
    for i in range(len(min_grads)):
        for j in range(len(layers)):
            if max_grads[i][j].item() > max_array[j]:
                max_array[j] = max_grads[i][j].item()
            
            mean_array[j] += ave_grads[i][j].item()
            
            if min_grads[i][j].item() < min_array[j]:
                min_array[j] = min_grads[i][j].item()
                   
    mean_array /= len(min_grads)
    
    print(len(layers))
    print(len(min_grads))
    print(len(max_grads))
    print(len(ave_grads))    

    print(min_array)
    print(mean_array)
    print(max_array)
    
    fig = go.Figure()
    df = pd.DataFrame(
#        {'mean': [0.1,0.4,0.5,0.6,0.7,0.8,0.9],
#        'max': [0.3,0.5,0.6,0.9,1.1,1.3,1.6],
#        'min': [0.0,0.2,0.4,0.3,0.5,0.2,0.3],
        {'mean': mean_array,
        'max': max_array,
        'min': min_array,         
        'labels': layers})

    fig.add_trace(go.Scatter(x=df['labels'], y=df['min'],
        fill=None,
        mode='lines',
        line_color=get_continuous_color(scale, 0),
        name="Min",
        line={'width': 4},
        legendgroup="group1"
        ))
    fig.add_trace(go.Scatter(
        x=df['labels'],
        y=df['mean'],
        legendgroup="group2",
        fill='tonexty',
        fillcolor=get_continuous_color(scale, 0.25).replace('rgb','rgba').replace(')',',0.5)'),
        line_color=get_continuous_color(scale, 0.25),
        mode='lines', name="Bottom Half"))
    fig.add_trace(go.Scatter(x=df['labels'], y=df['mean'],
        fill=None,
        mode='lines',
        legendgroup="group1",
        line={'width': 4},
        line_color=get_continuous_color(scale, 0.5),
        name="Mean"))
    fig.add_trace(go.Scatter(
        x=df['labels'],
        y=df['max'],
        legendgroup="group2",
        fill='tonexty',
        fillcolor=get_continuous_color(scale, 0.75).replace('rgb','rgba').replace(')',',0.5)'),
        line_color=get_continuous_color(scale, 0.75),
        mode='lines',
        name="Top Half"))
    fig.add_trace(go.Scatter(x=df['labels'], y=df['max'],
        fill=None,
        mode='lines',
        legendgroup="group1",
        line={'width': 4},
        line_color=get_continuous_color(scale, 1),
        name="Max"
        ))

    if display_plot:
        fig.show()
        
    if save_plot:
        fig.write_html(basefn + 'gradients.html')
    


In [ ]:
# train_loss and val_loss are lists containing the loss per step


def plot_losses(train_loss, val_loss, title='Train / Validation Loss', basefn='', display_plot=0, save_plot=1):

    iter = len(train_loss)
    
    # Plot the training / validation loss vs step
    trace1 = {
        'x': np.arange(0,len(train_loss)),
        'y': np.array(train_loss),
                  
        'type': 'scatter',
        'name': 'Training',
        'showlegend': True
    }

    trace2 = {
        'x': np.arange(0,len(val_loss)),
        'y': np.array(val_loss),
                  
        'type': 'scatter',
        'name': 'Validation',
        'showlegend': True
    }

    data = [trace1, trace2]
        
    layout = go.Layout({
        'title': {
            'text': title,
            'font': {
                'size': 20
            }
        }
    })

    fig = go.Figure(data, layout)
    
    if display_plot:
        fig.show()
        
    if save_plot:
        fig.write_html(basefn + 'Train_losses.html')
    



In [ ]:
def plot_returns_hist(predicted_returns, actual_returns, title='Histogram: Predicted vs Actual Returns', basefn='', display_plot=0, save_plot=1):

    fig = go.Figure()
    fig.add_trace(go.Histogram(x=predicted_returns, name='Predicted'))
    fig.add_trace(go.Histogram(x=actual_returns, name='Actual'))
    fig.update_layout(
        title_text = title,
        xaxis_title_text = 'Return',
        yaxis_title_text = 'Count'
    )
    
    if display_plot:
        fig.show()
    
    if save_plot:
        fig.write_html(basefn + 'Returns_hist.html')

In [ ]:
def plot_hidden(h, basefn='', display_plot=0, save_plot=1):
    
    # h is a tuple containing 2 tensors (one for h, one for c). 
    # Each tensor is of dimension [num layers, batch size, hidden size]
    # e.g. [3, 1, 128]
        
    titles = ['Hidden State', 'Cell State']
    
    x = np.arange(0, h[0].size()[2])
        
    # Loop through hidden and cell states
    for state in range(2):
        data = []
        # Loop through each layer, add a trace for each
        for layer in range(h[state].size()[0]):
            h_tensor = h[state][layer][0].cpu()
            trace = {
                'x': x,                
                'y': np.array(h_tensor.detach().numpy()),
                  
                'type': 'scatter',
                'name': 'Layer '+str(layer),
                'showlegend': True
                }
            data.append(trace)
        
        layout = go.Layout({
            'title': {
                'text': titles[state],
                'font': {
                    'size': 20
                }
            }
        })

        fig = go.Figure(data, layout)
        
        if display_plot:
            fig.show()
        
        if save_plot:
            fig.write_html(basefn + 'Train_hidden_layer_snapshot.html')


In [ ]:
def plot_lstm_output(lstm_output, basefn='', display_plot=0, save_plot=1):
    
    # lstm_output is a tuple containing [sequence length] number of tensors.
    # Each tensor is the lstm output with dimensions [batch size, hidden size]
    # e.g. [1, 128]
    
    x = np.arange(0, lstm_output.size()[1])
        
    data = []
    
    # Loop through each layer, add a trace for each
    for i in range(len(lstm_output)):
        lstm_tensor = lstm_output[i].cpu()
        trace = {
            'x': x,
            'y': np.array(lstm_tensor.detach().numpy()),
                  
            'type': 'scatter',
            'name': 'Cell '+str(i),
            'showlegend': True
            }
        data.append(trace)
        
    layout = go.Layout({
        'title': {
            'text': 'LSTM Output',
            'font': {
                'size': 20
            }
        }
    })

    fig = go.Figure(data, layout)
    
    if display_plot:
        fig.show()   
        
    if save_plot:
        fig.write_html(basefn + 'Train_LSTM_output_snapshot.html')
    

In [ ]:
def plot_weights(model, basefn='', display_plot=0, save_plot=1):
    
    # Plot the linear weights - just a simple line chart
    linear_tensor = model.linear.weight[0].cpu()
    x = np.arange(0, len(model.linear.weight[0]))
    y = np.array(linear_tensor.detach().numpy())
  
    # Plot the training / validation loss vs step
    trace1 = {
        'x': x,
        'y': y,
                  
        'type': 'scatter',
        'name': 'Linear Weights',
        'showlegend': True
    }

    data = [trace1]
        
    layout = go.Layout({
        'title': {
            'text': 'Linear Weights',
            'font': {
                'size': 20
            }
        }
    })

    fig = go.Figure(data, layout)
    
    if display_plot:
        fig.show()
        
    if save_plot:
        fig.write_html(basefn + 'Train_linear_weights.html')
    
    ave_weights = []
    max_weights = []
    layers = []
    for n,p in model.named_parameters():
        if (p.requires_grad) and ("bias" not in n) and ("linear" not in n):
            layers.append(n)
            ave_weights.append(p.abs().mean().item())
            max_weights.append(p.abs().max().item())

    trace1 = {
        'x' : layers,
        'y' : ave_weights,
        'type' : 'bar'
    }    

    data = [trace1]
    layout = go.Layout({
        'title': {
            'text': 'Average of LSTM Weights',
            'font': {
                'size': 20
            }
        }
    })

    fig = go.Figure(data, layout)

    if display_plot:
        fig.show()
        
    if save_plot:
        fig.write_html(basefn + 'Train_ave_lstm_weights.html')
    
    
    trace1 = {
        'x' : layers,
        'y' : max_weights,
        'type' : 'bar'
    }    

    data = [trace1]
    layout = go.Layout({
        'title': {
            'text': 'MAX of LSTM Weights',
            'font': {
                'size': 20
            }
        }
    })

    fig = go.Figure(data, layout)
    
    if display_plot:
        fig.show()
        
    if save_plot:
        fig.write_html(basefn + 'Train_max_lstm_weights.html')
        
    

In [ ]:
def gradient_plot(layer_names, grads, color, title, basefn='', display_plot=0, save_plot=1):
       
    for i in range(len(grads)):
        plt.plot(grads[i], alpha=0.3, color=color)
    
    plt.hlines(0, 0, len(grads)+1, linewidth=1, color="k")
    plt.xticks(range(0,len(grads[0]),1), layer_names, rotation="vertical")
    plt.xlim(xmin=0, xmax=len(grads[0]))
    
    plt.xlabel("Layers")
    plt.ylabel("Gradient")
    plt.title(title)
    plt.grid(True)
           
    # This has to go before the plt.show() otherwise it's blank
    if save_plot:
        plt.savefig(basefn + title + '.png', bbox_inches='tight')

    if display_plot:
        plt.show()


In [ ]:
def plot_gradients(model, ave_grads, max_grads, basefn='', display_plot=0, save_plot=1):

    print("Number of gradients randomly sampled:", len(ave_grads))    
    
    # Assemble the names of the layers
    layers = []
    for n,p in model.named_parameters():
        if (p.requires_grad) and ("bias" not in n):
            layers.append(n)

    gradient_plot(layers, ave_grads, 'b', 'Gradient Flow - All Layers - Average', basefn=basefn, display_plot=display_plot, save_plot=save_plot)
    gradient_plot(layers, max_grads, 'r', 'Gradient Flow - All Layers - Max', basefn=basefn, display_plot=display_plot, save_plot=save_plot)
        
    ave_grads_lstm = [sublist[:len(layers)-1] for sublist in ave_grads]
    max_grads_lstm = [sublist[:len(layers)-1] for sublist in max_grads]
    layers_lstm = layers[0:len(layers)-1]
    
    gradient_plot(layers_lstm, ave_grads_lstm, 'b', 'Gradient Flow - LSTM Layers - Average', basefn=basefn, display_plot=display_plot, save_plot=save_plot)
    gradient_plot(layers_lstm, max_grads_lstm, 'r', 'Gradient Flow - LSTM Layers - Max', basefn=basefn, display_plot=display_plot, save_plot=save_plot)
 


In [ ]:

class NeilLSTM(nn.Module):
    def __init__(self, input_size = 6, window_size = 11, arch = 'LH', hidden_size = 64, output_size = 1, num_layers = 3, dropout=0.5):
        super().__init__()
        
        # Store some stuff we need to know in the forward pass
        self.n_layers = num_layers        
        self.hidden_layer_size = hidden_size
        self.arch = arch

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, 
                            batch_first=False, bias=True, dropout=dropout, bidirectional=False)
                
        # Define the linear layer
        # Perhaps should perform forward on the lstm to explicitly get the size of the 
        # tensor for sequence to one vs sequence to sequence versions.
        
        # The size of the linear layer for LH or AH model
        if arch == 'LH':
            self.linear = nn.Linear(hidden_size, output_size, bias = True)
        elif arch == 'AH':
            self.linear = nn.Linear(hidden_size * window_size, output_size, bias = True)
        
        torch.nn.init.xavier_uniform_(self.linear.weight)
        
        # NEED TO FIX THIS STUPID HARD CODED INIT
        if num_layers >= 1:
            torch.nn.init.xavier_uniform_(self.lstm.weight_hh_l0)
            torch.nn.init.xavier_uniform_(self.lstm.weight_ih_l0)
            
        if num_layers >= 2:
            torch.nn.init.xavier_uniform_(self.lstm.weight_hh_l1)        
            torch.nn.init.xavier_uniform_(self.lstm.weight_ih_l1)                    
        
        if num_layers >= 3:
            torch.nn.init.xavier_uniform_(self.lstm.weight_hh_l2)                        
            torch.nn.init.xavier_uniform_(self.lstm.weight_ih_l2)                        
        
        
    # Forward pass
    # input_seq shape is: (sequence length, num input features) - there's no batch dimension
    def forward(self, input_seq, hidden, verbose=0):

        if verbose:
            print("\n\nlstm input:\n", input_seq)
            print("\n\nlstm input view:\n", input_seq.view( len(input_seq) , 1 , -1))
            print("\n\nlstm hidden input:\n", hidden)
            
        lstm_out, hidden = self.lstm( input_seq.view( len(input_seq) , 1 , -1), hidden)
        
        if verbose:
            print("\n\nlstm_output:\n", lstm_out)
            print("\n\nhidden output:\n", hidden)
                    
        # Feed forward the outputs of the lstm into the linear layer       
        # lstm_out dims are (seq_len, batch, hidden dim)
        # This is converted to (seq_len, hidden_dim) before input to the linear layer        
        # When T=22 and hidden_layer_size=64, the shape of the lstm_out tensor is [22,1,64]    
        # This view reshapes it to [22, 64]
        lstm_out_view = lstm_out.view(len(input_seq), -1)
        
        if verbose:
            print("\n\nlstm_out_view:\n", lstm_out_view)
                                       
        if verbose:
            print("\n\nLinear weights:\n", self.linear.weight)
            print("\n\ninput to fc:\n", lstm_out_view[-1])
        
        # LH - Just present the last hidden state to the linear network            
        if self.arch == 'LH':
            predictions = self.linear(lstm_out_view[-1])
        # AH - flatten and present all to linear network
        elif self.arch == 'AH':
            predictions = self.linear(torch.flatten(lstm_out_view))
        
        if verbose:
            print("\n\npredictions:\n",predictions)
        
        # predictions has dimension: (1, seq_len)        
        return predictions, hidden, lstm_out_view
    
    
    def init_hidden(self, batch_size = 1):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_layer_size).zero_().to(device),
                 weight.new(self.n_layers, batch_size, self.hidden_layer_size).zero_().to(device))
        return(hidden)
        

In [139]:
# Trains the given LSTM model. Also collects validation data (predicting the next day) and collects stats
# for both.  These are used to observe training vs validation but also to construct the first allocation policy.

# Q version has removed inner loop instrumentation and made other spped changes
# This version runs about 20% faster than the non-Q version

def train_modelQ(model, train_seq, train_labels, scale=1, offset=1, train_start=0, train_length=1, epochs = 20, iter_per_seq=1, max_iter=1600, lr = 0.001, dr=0.999, wd=0.0, verbosity=0, basefn='',percentiles=[0,10,20,30,40,50,60], trading_validation_period=100, valid_seq=[], valid_labels=[], plot=0):
    input_size = 6
    
    
    # if plot level is 4, plot graphs on screen for every inner loop training run
    if plot == 4:
        dump_log_interval = 1
    else:
        dump_log_interval = 10 # otherwise do it every 10
        
    loss_function = nn.MSELoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay = wd)
    my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer = optimizer, gamma = dr)
    
    batch_size = 1  # batch mode isn't used actually but including for future possible use
    
    with open(basefn + 'train.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(['Epc','AvTrLoss','AvTrMSE','AvValMSE','MdnTrMSE','MdnValMSE','ValMDA','ValMAPE','ValMAE','PXcorr','RXcorr',
                     'EpcSecs','TtlMins'])
        
    with open(basefn + 'trade_validation.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(['Epoch','MDA','MSE','MAE','MAPE','X-Correl','Num-trades','AveRt%','MdnRT%','MinRt%','MaxRt%',
                    'RangeRt%','StdDevRt%'])
            
    # Outputs a summary line for each Epoch
    if verbosity == 1:
        print('Epc  AvTrLoss  AvTrMSE   AvValMSE  MdnTrMSE  MdnValMSE  ValMDA    ValMAPE  ValMAE    PXcorr  RXcorr Secs    TTlMins')

    # Outputs a summary line for each sequence in each epoch
    if verbosity == 2:
        print('EPc  Seq#   TrLoss    TrMSE     PredR   ActRt   ValLoss   VAdjC    VPred   VAct      VPredRt   VActRt   RunMDA')            
                            
    
#    train_losses = []
#    val_adj_close_price = []
    ave_grads = []
    max_grads = []
    min_grads = []
    
    # If the number of epochs is high, just sample the gradients stochastically to avoid retaining too much
    # data.
    if (epochs * train_length) > 1000:
        sample_grads = 1000.0 / (epochs * train_length)
    else:
        sample_grads = 1.0

    start_time = time.time()       
    
    train_seq_cuda = train_seq.to(device)
    train_labels_cuda = train_labels.to(device)

    # How many times through the entire training set
    for i in range(epochs):
        train_losses = []
        val_adj_close_price = []        
        train_mses = []
        val_mses = []
        val_pred_rt = []
        val_act_rt = []
        val_pred_price = []
        val_act_price = []        
        running_mda = 0
        epoch_start_time = time.time()
        
        model.train()
        
        # Each sample in the training set
        for j in range(train_start, train_start + train_length):      
                        
            inputs = train_seq_cuda[j]
            labels = train_labels_cuda[j]
            
            # How many times we train on each sample as we go
            for iter in range(iter_per_seq):
                # Initialise hidden states before every training event
                h = model.init_hidden(batch_size)  
                                    
                optimizer.zero_grad()
        
                y_pred, h, lstm_out = model(inputs, h)
            
                # y_pred and labels are both size T - sequence to sequence loss function
                train_loss = loss_function(y_pred, labels)
            
                # Compute gradients and update the model
                train_loss.backward()
                                
                optimizer.step()
                
            # For speed, we only do outer loop validation at the same time we do trading validation (the stats are
            # needed to compute the initial allocation policy)
            
            if i % trading_validation_period == 0:
                # Sample stochastically the last set of gradients after the inner loop for plotting later
                if plot and (random.random() < sample_grads):
                    ave_grad=[]
                    max_grad=[]
                    min_grad=[]
                    for n,p in model.named_parameters():
                        if (p.requires_grad) and ("bias" not in n):
                            ave_grad.append(p.grad.abs().mean())
                            max_grad.append(p.grad.abs().max())
                            min_grad.append(p.grad.abs().min())
                    ave_grads.append(ave_grad)
                    max_grads.append(max_grad)
                    min_grads.append(min_grad)            
            
                # Capture the (snapshot) metrics from the last inner loop cycle
                training_loss = train_loss.item()  # This is the sequence to sequence training loss                        
                train_adj_close = inputs[-1][-1].item()   # Adj close at t=0
                train_pred = y_pred[-1].item()  # Next day predicted Adj close
                train_truth = labels[-1].item()  # Next day actual Adj close
                train_mse = (train_pred - train_truth)**2   # MSE of just the predicted value vs truth
                train_predicted_return = (denormalise_value(train_pred, scale, offset) / denormalise_value(train_adj_close, scale, offset)) - 1            
                train_actual_return = (denormalise_value(train_truth, scale, offset) / denormalise_value(train_adj_close, scale, offset)) - 1

                # Outer loop validation every trading_validation_period epochs
                model.eval()
            
                inputs = train_seq[j+1].to(device)
                labels = train_labels[j+1].to(device)
            
                with torch.no_grad():
                    h = model.init_hidden(batch_size)
        
                    y_pred, h, lstm_out = model(inputs, h)
                    
                    valid_adj_close = inputs[-1][-1].item()                
                    valid_pred = y_pred[-1].item()
                    valid_truth = labels[-1].item()
                    valid_loss = (valid_pred - valid_truth)**2  # Just the mse of the valid predicted value vs truth
            
                    valid_predicted_return = (denormalise_value(valid_pred, scale, offset) / denormalise_value(valid_adj_close, scale, offset)) - 1
                    valid_actual_return = (denormalise_value(valid_truth, scale, offset) / denormalise_value(valid_adj_close, scale, offset)) - 1

                    if (valid_predicted_return * valid_actual_return >= 0):
                        running_mda += 1                
                
                    # If enabled, print results line at the end of inner loop training on each sequence
                    if (verbosity==2):
                        print('{:4}'.format(i),
                            '{:4}'.format(j), 
                          '  {:1.1e}'.format(training_loss),
                          '  {:1.1e}'.format(train_mse),
                          ' {: 2.3f}'.format(100.0*train_predicted_return), 
                          ' {: 2.3f}'.format(100.0*train_actual_return),
                          '  {:1.1e}'.format(valid_loss), 
                          ' {: 1.4f}'.format(valid_adj_close), 
                          ' {: 1.4f}'.format(valid_pred), 
                          ' {: 1.4f}'.format(valid_truth),
                          ' {: 2.4f}'.format(100.0*valid_predicted_return), 
                          ' {: 2.4f}'.format(100.0*valid_actual_return),
                          '  {:2.2f}'.format(100.0*running_mda/(j+1)))
                    
                    # Keep lists of the results of train / validation for each sequence for passing back up the stack
                    train_losses.append(training_loss)
                    train_mses.append(train_mse)
                    val_mses.append(valid_loss)
                    val_pred_rt.append(valid_predicted_return)
                    val_act_rt.append(valid_actual_return)
                    val_pred_price.append(valid_pred)
                    val_act_price.append(valid_truth)
                    val_adj_close_price.append(valid_adj_close)
                    
                model.train()
                
            # End of outer loop
            
        my_lr_scheduler.step()            
           
        elapsed_secs = time.time()-start_time
        print(i, time.time()-epoch_start_time, elapsed_secs, optimizer.param_groups[0]['lr'])
        
        if i % trading_validation_period == 0:
            # We report stats on the last portion of the training
            stats_len = int(0.5 * len(train_losses))
            
            metrics_dict = compute_metrics(np.array(val_pred_rt[-stats_len:]),
                                   np.array(val_act_rt[-stats_len:]),
                                   np.array(val_pred_price[-stats_len:]),
                                   np.array(val_act_price[-stats_len:]))

            # This is the results we'll pass back up the stack for the entire training
            result_dict = {'trStats_N':stats_len, 'AvTrLoss':np.mean(train_losses[-stats_len:]),
                   'trAvTrMSE':np.mean(train_mses[-stats_len:]),
                   'trAvValMSE':np.mean(val_mses[-stats_len:]),
                   'trMdnTrMSE':np.median(train_mses[-stats_len:]),
                   'trMdnValMSE':np.median(val_mses[-stats_len:]),
                   'train_mses':train_mses, 'valid_mses':val_mses, 'val_pred_rt':val_pred_rt, 'val_act_rt':val_act_rt, 
                   'val_pred_price':val_pred_price, 'val_act_price':val_act_price, 
                   'val_adj_close_price': val_adj_close_price,
                    'trMDA': metrics_dict['MDA'],
                    'trMAPE': metrics_dict['MAPE'],
                    'trMAE': metrics_dict['MAE'],
                    'trPXCORR': metrics_dict['PXCORR'],
                    'trRXCORR': metrics_dict['RXCORR'],
                    'epoch_train_time':time.time()-epoch_start_time,
                   'train_time':elapsed_secs}
        
            # Output summary stats at end of outer loop training  
            if (verbosity == 1):
                print('{:4}'.format(i),'{:4.2e}'.format(result_dict['AvTrLoss']),
                          ' {:1.2e}'.format(result_dict['trAvTrMSE']),
                          ' {:4.2e}'.format(result_dict['trAvValMSE']), 
                          ' {:1.2e}'.format(result_dict['trMdnTrMSE']),
                          ' {:4.2e}'.format(result_dict['trMdnValMSE']),                  
                          '  {:2.2f}'.format(result_dict['trMDA']), 
                          '    {:2.2f}'.format(result_dict['trMAPE']), 
                          '    {:2.2e}'.format(result_dict['trMAE']),
                          '{: 1.2f}'.format(result_dict['trPXCORR']), 
                          '  {: 1.2f}'.format(result_dict['trRXCORR']),      
                          '  {:2.2f}'.format(result_dict['epoch_train_time']),                  
                          '  {:3.1f}'.format(result_dict['train_time']/60.0))
            
            with open(basefn + 'train.csv', 'a', newline='') as myfile:
                wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                wr.writerow([i, result_dict['AvTrLoss'],
                        result_dict['trAvTrMSE'],
                        result_dict['trAvValMSE'],
                        result_dict['trMdnTrMSE'],
                        result_dict['trMdnValMSE'],
                        result_dict['trMDA'],
                        result_dict['trMAPE'],
                        result_dict['trMAE'],
                        result_dict['trPXCORR'],
                        result_dict['trRXCORR'],
                        result_dict['epoch_train_time'],
                        result_dict['train_time']/60.0,
                        ])
        
        # Periodically perform a trading validation 
        if i % trading_validation_period == 0:
            
            # Make a results directory. If the 'directory' doesn't exist, create it
            if os.path.isdir(basefn + '/saved_models') == False:
                os.mkdir(basefn + '/saved_models')
            
            D,Q,E,A,C = generate_initial_allocation_policy(result_dict, percentiles=percentiles, scale=scale, offset=offset, basefn=basefn + 'saved_models/'+str(i), verbosity=0)
            
            # Save both the model and the initial allocation policy so we can recreate for validation / test after training
            # (pick the best model out of a complete training run after analysing the data)
                        
            torch.save(model,basefn + '/saved_models/epoch' + str(i) + '.mdl')
            
            with open(basefn + '/saved_models/DQEAC_dict' + str(i) + '.pkl','wb') as fp:
                pickle.dump({'D':D, 'Q':Q, 'E':E, 'A':A, 'C':C}, fp, protocol=pickle.HIGHEST_PROTOCOL)            

            # Note - prior to version 15, I wasn't deep copying the D,Q,E,A which means they would have been being updated
            # by trading validation.
            trial_results = []
            for trial in range(5):
                # For each trial make a deep copy of things that get modified by the validation process. Make sure that 
                # the model being trained and the allocation policy components are not being modified and we always start
                # validation from the same starting point.
                validation_model = copy.deepcopy(model)
                copy_D = copy.deepcopy(D)
                copy_Q = copy.deepcopy(Q)
                copy_E = copy.deepcopy(E)
                copy_A = copy.deepcopy(A)
                
                validation_model, copy_D, copy_Q, copy_E, copy_A, journal, test_result_dict = test_trading_system(validation_model, valid_seq, valid_labels, copy_D, copy_Q, copy_E, copy_A, percentiles, basefn=basefn+'/saved_models/'+str(i)+'_'+str(trial), scale=scale, offset=offset, iter_per_seq=iter_per_seq, lr = 0.001, dr = 1, verbosity=0)
                
                trial_results.append(test_result_dict)                

            
            mean_trial_MDA = np.mean([v['MDA'] for v in trial_results])
            mean_trial_MSE = np.mean([v['MSE'] for v in trial_results])
            mean_trial_MAE = np.mean([v['MAE'] for v in trial_results])
            mean_trial_MAPE = np.mean([v['MAPE'] for v in trial_results])
            mean_trial_RXCORR = np.mean([v['RXCORR'] for v in trial_results])
            mean_trial_num_trades = np.mean([v['num_trades'] for v in trial_results])
            trial_returns = [v['trade_return'] for v in trial_results]
            mean_trial_trade_return = np.mean(trial_returns)
            median_trial_trade_return = np.median(trial_returns)
            min_trial_trade_return = np.min(trial_returns)
            max_trial_trade_return = np.max(trial_returns)
            range_trial_trade_return = max_trial_trade_return - min_trial_trade_return
            std_trial_trade_return = np.std(trial_returns)
            
            print('Trading Validation - ',
                'MDA:{:2.2f}'.format(mean_trial_MDA),
                '  Returns Correl:{:2.2f}'.format(mean_trial_RXCORR),
                '  Num Trades:{:4}'.format(mean_trial_num_trades),
                '  Return%:{:2.2f}'.format(mean_trial_trade_return))
            
            with open(basefn + 'trade_validation.csv', 'a', newline='') as myfile:
                wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                wr.writerow([i,mean_trial_MDA, mean_trial_MSE,
                            mean_trial_MAE, mean_trial_MAPE,
                            mean_trial_RXCORR, mean_trial_num_trades, 
                            mean_trial_trade_return, median_trial_trade_return,
                            min_trial_trade_return, max_trial_trade_return,
                            range_trial_trade_return, std_trial_trade_return])

    if plot>=1:
        plot_gradients(model, ave_grads, max_grads, basefn=basefn, display_plot=plot, save_plot=1)
        plotly_gradients(model, min_grads, max_grads, ave_grads, basefn=basefn, display_plot=plot, save_plot=1)
        plot_hidden(h, basefn=basefn, display_plot=plot, save_plot=1)
        plot_lstm_output(lstm_out, basefn=basefn, display_plot=plot, save_plot=1)
        plot_weights(model, basefn=basefn, display_plot=plot, save_plot=1)
        plot_losses(train_mses, val_mses, title='Model Training: Training Loss vs Validation Loss', basefn=basefn, display_plot=plot, save_plot=1)
        plot_returns_hist(val_pred_rt, val_act_rt, title='Model Training: Predicted vs Actual Returns', basefn=basefn+'both_', display_plot=plot, save_plot=1)
        plot_returns_hist(val_pred_rt, val_pred_rt, title='Model Training: Predicted Returns', basefn=basefn+'pred_', display_plot=plot, save_plot=1)
        plot_returns_hist(val_act_rt, val_act_rt, title='Model Training: Actual Returns', basefn=basefn+'act_', display_plot=plot, save_plot=1)    
            
    return model, result_dict



In [140]:
# This creates a model instance and trains it on the training set

def create_train_model(df, resume_dir='', resume_epoch=-1, basefn='', scale=1, offset=1, arch='LH', window_size = 22, epochs=20, iter_per_seq=1, max_iter=1600, num_layers=2, hidden_size=64, dropout=0.5, lr=0.001, dr=1, wd=0.0, percentiles=[0,10,20,30,40,50,60], trading_validation_period = 100, valid_seq=[], valid_labels=[], verbosity=0, plot=0):

    # Generate training set
    train_seq, train_labels = create_input_sequences_from_df(df, window_size)

    # Convert our list of lists to Tensors
    train_seq = torch.FloatTensor(train_seq)
    train_labels = torch.FloatTensor(train_labels)
       
    if verbosity>=1:
        print("Building LSTM Model with the following parameters:")
        print("Architecture:", arch)
        print("Num layers:", num_layers)
        print("Hidden size:", hidden_size)
        print("Dropout:", dropout)
        print("Learning Rate:", lr)
        print("LR Decay Rate:", dr)
        print("Weight Decay:", wd)
        print("Window size:", window_size)
        print("Num Epochs:", epochs)                
        print("Iterations per seq:", iter_per_seq)
        print("Start training...")

           
    start_time = time.time()
    
    # If we are starting from scratch, create a brand new model
    if resume_dir=='':
        model = NeilLSTM(input_size=6, window_size=window_size, arch=arch, hidden_size=hidden_size, output_size=window_size, num_layers=num_layers, dropout=dropout)
        last = 'NONE'
    else: # Otherwise, load it from disk
        # Special case, load the last model in directory
        if resume_epoch == -1:
            files = glob.glob(resume_dir + '/saved_models/*.mdl')
            last = max(files, key=os.path.getctime)
            model = torch.load(last)
            if verbosity >= 1:
                print("Loaded existing model:", last)

# Dump the model configuration and results to log file
    with open(basefn + 'train.csv', 'w', newline='') as myfile:
        
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(['Norm Scale:', scale])
        wr.writerow(['Norm Offset:', offset])
        wr.writerow(['Arch:', arch])
        wr.writerow(['Window Size:', window_size])
        wr.writerow(['Hidden Size:', hidden_size])
        wr.writerow(['Num Layers:', num_layers])        
        wr.writerow(['Dropout:', dropout])                
        wr.writerow(['Learning Rate:', lr]) 
        wr.writerow(['LR Decay Rate:', dr])
        wr.writerow(['Weight Decay:', wd])        
        wr.writerow(['Num Epochs:', epochs])
        wr.writerow(['Iterations Per Sequence:', iter_per_seq])
        wr.writerow(['Start Epoch:', resume_epoch])
        wr.writerow(['Resume-from model path:', last])
        wr.writerow(['Training Start Date:', df['Date'].iloc[0]])
        wr.writerow(['Training End Date:', df['Date'].iloc[-1]])                    
                
    model.to(device)

    # Stop one short of the end as we use the t+1 for validation during training
    model, result_dict = train_modelQ(model, train_seq, train_labels, scale=scale, offset=offset, train_start=0, 
                                     train_length=len(train_seq)-1, epochs=epochs, iter_per_seq=iter_per_seq, 
                                     max_iter=max_iter, lr=lr, dr=dr, wd=wd, verbosity=verbosity, basefn=basefn, 
                                     plot=plot, trading_validation_period=trading_validation_period, 
                                     percentiles=percentiles, valid_seq=valid_seq, valid_labels=valid_labels)
        
    # Save the model with the unique filename
    torch.save(model,basefn + '_train.mdl')
    
    return model, result_dict

In [141]:
def compute_norm_xcorrel(a, b):
    c = np.corrcoef(a, b)

    return c[0][1]

In [142]:
# Compute the interim metrics per the paper table 8 - MDA, MAPE, MAE, MSE
# Also added cross-correlation

# Pass in lists of next day predicted and actual returns, predicted and actual prices
def compute_metrics(pred_rt, act_rt, pred_price, act_price):
    
    len_data = len(pred_rt)

    product = pred_rt * act_rt
    abs_price_diff = np.abs(act_price - pred_price)
    
    MDA = 100 * len(np.where(product > 0)[0]) / len_data
    MAPE = 100 * np.sum(abs_price_diff / np.abs(act_price)) / len_data
    MAE = np.sum(abs_price_diff) / len_data
    MSE = np.sum(abs_price_diff ** 2) / len_data
    
    price_xcorrel = compute_norm_xcorrel(pred_price, act_price)
    returns_xcorrel = compute_norm_xcorrel(pred_rt, act_rt)
    
    metrics_dict = {'N':len_data, 'MDA':MDA, 'MAPE': MAPE, 'MAE': MAE, 'MSE':MSE, 'PXCORR':price_xcorrel, 
                    'RXCORR':returns_xcorrel}
    
    return metrics_dict

In [143]:
# Call this routine whenever a new predicted return is generated (each new day)

# inputs:
# D - a list of all the predicted returns, by trading day, to date
# Q - a list of the bin cut-offs
# new_pred_return - the new predicted return from the model
# percentiles - a list of percentiles used to determine the bins (e.g. [0,10,20,30,40,50,60])

# returns:
# D - Updated
# Q - updated

def update_DQ(D, Q, new_pred_return, percentiles):

    # Append the new predicted return to the existing list
    D = np.append(D,new_pred_return)
    
    # If the predicted return is negative, it won't affect the binning calculation
#    if new_pred_return <= 0:
#        return D, Q
    
    # Percentile the positive returns
    returns = np.array(D)
#    Q = np.percentile(returns[returns >= 0], percentiles)
    Q = np.percentile(abs(returns), percentiles)
    
    # Q now contains the cutoff values for each bin.  Each bin will contain approximately 10% of the positive
    # returns except the last one which is an amalgamation of 4 bins so it will contain approx 40%
        
    # return updated D and Q
    return D, Q

In [144]:
# Call this routine whenever a new sell is executed (and therefore we have a new expected value for a bin)

def update_EA(E, A, profit, purchase_bin):

    # Append the new profit amount to the correct bin in E        
    E[purchase_bin] = E[purchase_bin] + profit
    
    # Update A - bins with positive total profits are set to 1 (a buy signal)
    # bins with negative profits are set to 0 - a do nothing signal
    # bin 0 is the unique, sell signal
    A = (E > 0).astype(int)    
    
    return E, A

    

In [174]:
# This function combines model training (updates), predicting next day return, trading and updating the trading
# policy each day.
# Much of the code is a repeat / combo of the above two seperate routines of training the model and 
# genertaing the policy. This is different as we are now "live" and updating the model / policy only on
# previously unseen data.
# Also, although the model is using Adj Close as the label for training, what we need to record for evaluation
# is the Open price for purchases and sales (assuming we are running the model overnight between two trade
# days).The model updates and issues trades which will be filled at the next trading day open.

# predictor can be "pytorch_lstm", "random", "prescient", "drunk_genius"

def test_trading_system(model, test_seq, test_labels, D, Q, E, A, percentiles, predictor="pytorch_lstm", drunk_genius_accuracy=0.5, basefn='', scale=1, offset=1, iter_per_seq=1, lr = 0.001, dr = 1, verbosity=0):
    input_size = 6
        
    print(basefn)
    
    loss_function = nn.MSELoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer = optimizer, gamma = dr)
    
    batch_size = 1  # batch mode isn't used actually but including for future possible use
            
    holding = False;
    journal = []   
    total_profit_open = 0
    total_profit_adj_close = 0
    C = np.zeros(shape=(len(E)))
    
    # Keep list of predicted and actual returns and prices (for metrics reporting)
    test_pred_rt = []
    test_act_rt = []
    test_pred_price = []
    test_act_price = []    
    
    start_time = time.time()
    
    trades_fn = basefn + '_trades.csv'
    losses_fn = basefn + '_losses.csv'
    
    # Write headings for trades file
    with open(trades_fn, 'w', newline='') as myfile:    
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(['Day','Adj Close','Pred Rt', 'Act Rt','Buy/Sell','Adj Cls Sell Price','Open Sell Price',
                     'Bin','AdjCls Profit','Open Profit','Acc Profit Adj Cls','Acc Profit Open',
                     'E0','E1','E2','E3','E4','E5','E6','E7',
                     'A0','A1','A2','A3','A4','A5','A6','A7',                     
                     'C0','C1','C2','C3','C4','C5','C6','C7',
                     'Q0','Q1','Q2','Q3','Q4','Q5','Q6'])

    # Write headings for losses file
    with open(losses_fn, 'w', newline='') as myfile:    
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        heading_list=['Day','Trainloss','TestPred', 'TestAct', 'TestPredRt', 'TestActRt', 'TestLoss']
        for i in range(len(test_seq[0])):
            heading_list.append('TP'+str(i))
        for i in range(len(test_seq[0])):
            heading_list.append('TL'+str(i))            
        for i in range(len(test_seq[0])):
            heading_list.append('VP'+str(i))            
        for i in range(len(test_seq[0])):
            heading_list.append('VL'+str(i))                        
        wr.writerow(heading_list)
        
    total_profit = 0
    num_trades = 0
    
    for j in range(0, len(test_seq) - 2):      

        # Send input / label data to Cuda if available
        train_inputs = test_seq[j].to(device)
        train_labels = test_labels[j].to(device)
                
#        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#        my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer = optimizer, gamma = dr)
                

        test_log = []
        
        # Reset the learning rate each time for the inner loop
#        optimizer.param_groups[0]['lr'] = lr        

        model.train()

        # How many times we train on each sample as we go
        for iter in range(iter_per_seq):

            # Initialise hidden states before every epoch
            h = model.init_hidden(batch_size)  
                                    
            optimizer.zero_grad()
        
            train_y_pred, h, lstm_out = model(train_inputs, h, verbose=0)
               
            train_test_loss = loss_function(train_y_pred, train_labels)
               
            # Compute gradients and update the model
            train_test_loss.backward()
            optimizer.step()
            my_lr_scheduler.step()
                       
        # Having updated the model with "todays" inputs, we now predict "tomorrows" return
        model.eval()
            
        inputs = test_seq[j+1].to(device)
        labels = test_labels[j+1].to(device)
            
        with torch.no_grad():
            h = model.init_hidden(batch_size)
        
            y_pred, h, lstm_out = model(inputs, h)
            test_adj_close = inputs[-1][-1].item()                
            test_pred = y_pred[-1].item()

            test_truth = labels[-1].item()
            test_actual_return = (denormalise_value(test_truth, scale, offset) / denormalise_value(test_adj_close, scale, offset)) - 1            
            
            # All trades are predicated on "test_predicted_return".  We mess with that here to implement the different kinds
            # of predictor - "pytorch_lstm", "random" or "prescient". We save the actual predicted return though and update
            # Q & Q with that, not the messed with value.
            
            pytorch_lstm_test_predicted_return =  (denormalise_value(test_pred, scale, offset) / denormalise_value(test_adj_close, scale, offset)) - 1
            
            if predictor == "pytorch_lstm":  # Usual behavior
                test_predicted_return = pytorch_lstm_test_predicted_return
            elif predictor == "random":
                test_predicted_return = random.choice(D)   # pick a random return from D
            elif predictor == "prescient":
                test_predicted_return = test_actual_return    # Make the predicted return the actual return
            elif predictor == "drunk_genius":   # Mix of prescient / random according to drunk_genius_accuracy
                if random.random() <= drunk_genius_accuracy:
                    test_predicted_return = test_actual_return
                else:
                    test_predicted_return = random.choice(D)   # pick a random return from D                

            test_pred_price.append(test_pred)
            test_act_price.append(test_truth)            
            test_loss = (test_pred - test_truth)**2
            
            test_pred_rt.append(test_predicted_return)
            test_act_rt.append(test_actual_return)
            
        if verbosity >= 3:
            print(j, '{:1.2f}'.format(test_adj_close), '{:1.4f}'.format(test_predicted_return))
            
        test_log = [j, test_adj_close, test_predicted_return, test_actual_return]
            
        # Sell if return is negative or if it's the last day of trading and we're holding the asset
        if ((test_predicted_return < 0 or j == len(test_seq) - 3)) and holding:
            open_sell_price = denormalise_value(test_seq[j+2][-1][0].item(), scale, offset) # Next day open
            sell_price = denormalise_value(test_adj_close, scale, offset)
            profit = sell_price - purchase_price
            total_profit += profit
            num_trades += 1
            E, A = update_EA(E, A, profit, purchase_bin)
            if verbosity >= 3:
                print("E", E)
                print("A", A)
            holding = False
            
            total_profit_open = total_profit_open + (open_sell_price - open_purchase_price)
            total_profit_adj_close = total_profit_adj_close + profit
            journal.append((-1, j+2+len(inputs), sell_price, total_profit_adj_close, open_sell_price, total_profit_open))
            
            if verbosity >= 2:
                print("Executed sell at ${:1.2f}".format(sell_price), "on day", j, "Profit: ${:1.4f}".format(profit),
                     "Total Profit: ${:1.4f}".format(total_profit))
                
            test_log += (['Sell', sell_price, open_sell_price, purchase_bin, profit, open_sell_price - open_purchase_price, 
                          total_profit_adj_close, total_profit_open] + E.tolist() + A.tolist() + C.tolist() + Q.tolist())

        # Buy
        if test_predicted_return >= 0 and not holding:
            return_bin = np.digitize(test_predicted_return, Q)
            if verbosity >= 3:
                print("Q",Q,"return_bin", return_bin)
            if A[return_bin] > 0:
                purchase_bin = return_bin
                C[purchase_bin] = C[purchase_bin] + 1
                num_trades += 1
                open_purchase_price = denormalise_value(test_seq[j+2][-1][0].item(), scale, offset) # Next day open
                purchase_price = denormalise_value(test_adj_close, scale, offset)
                holding = True
                journal.append((1, j+2+len(inputs), purchase_price, total_profit_adj_close, open_purchase_price, total_profit_open))
                
                if verbosity >= 2:
                    print("Executed buy at ${:1.2f}".format(purchase_price), "on day ", j, "bin", return_bin)
                    
                test_log += (['Buy', purchase_price, open_purchase_price, purchase_bin, 'N/A', 'N/A', 
                              total_profit_adj_close, total_profit_open] + E.tolist() + A.tolist() + C.tolist() + Q.tolist())

        D, Q = update_DQ(D, Q, pytorch_lstm_test_predicted_return, percentiles)
        
        # Append the metrics to the training log file
        with open(trades_fn, 'a', newline='') as myfile:    
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerow(test_log)
            
        # Make a record of the training / test losses
        with open(losses_fn, 'a', newline='') as losses_file:
            wr = csv.writer(losses_file, quoting=csv.QUOTE_ALL)
            wr.writerow([j,train_test_loss.item(),test_pred, test_truth, 
                         test_predicted_return, test_actual_return, test_loss] 
                        + train_y_pred.tolist() + train_labels.tolist() + y_pred.tolist() + labels.tolist())
        

    # The following is using Close - Open prices as start / end prices which is wrong and in all versions prior to 17
    # Item 0 is the Open price
#    start_price = denormalise_value(test_seq[0][-1][0].item(), scale, offset)

    # Item 3 is the Close price
#    end_price = denormalise_value(test_seq[-1][-1][3].item(), scale, offset)

    # First row, Last item is the Adj Close 
    start_price = denormalise_value(test_seq[0][-1][-1].item(), scale, offset)

    # Last row, Last item is the Adj Close
    end_price = denormalise_value(test_seq[-1][-1][-1].item(), scale, offset)
    
    if len(journal):
        test_profit = journal[-1][-3]
        test_return_pct = 100 * journal[-1][-3]/start_price
    else:
        test_profit = 0
        test_return_pct = 0
    
    if verbosity >= 1:

        print("Buy and Hold Baseline:")
        print("Price at start: ${:1.2f}".format(start_price))
        print("Price at end: ${:1.2f}".format(end_price))
        print("Difference: ${:1.2f}".format(end_price - start_price))
        print("Return:{:1.2f}%".format(100*(end_price - start_price)/start_price))

        print("\nProfit from trading:{:1.2f}".format(test_profit))
        print("Return from trading:{:1.2f}%".format(test_return_pct))
        print("Num trades:{:3}".format(num_trades))
    
        print("Length Returns:", len(D))
        print("Q:\n", Q)
        print("E:\n", E)
        print("A:\n", A)
        print("C:\n", C)
        
        print("Test Time:", (time.time()-start_time)/60)
        
    metrics_dict = compute_metrics(np.array(test_pred_rt),
                                   np.array(test_act_rt),
                                   np.array(test_pred_price),
                                   np.array(test_act_price))

    if verbosity >= 1:
        print("N:", metrics_dict['N'], 
              "\tMDA:{:1.2f}".format(metrics_dict['MDA']), 
              "\tMAPE:{:1.2f}".format(metrics_dict['MAPE']), 
              "\tMAE:{:1.5e}".format(metrics_dict['MAE']), 
              "\tMSE:{:1.5e}".format(metrics_dict['MSE']),
              "\tPrice Correlation:{:1.2f}".format(metrics_dict['PXCORR']),          
              "\tReturns Correlation:{:1.2f}".format(metrics_dict['RXCORR'])
             )
        
    result_dict = {'bnh_profit': end_price - start_price, 'bnh_Return':100*(end_price - start_price)/start_price,
              'num_trades':num_trades, 
              'trade_profit':test_profit, 'trade_return': test_return_pct, 'pred_rt':test_pred_rt, 
                  'act_rt':test_act_rt, 'pred_price':test_pred_price, 'act_price':test_act_price}

    # Append the metrics to the training log file
    with open(trades_fn, 'a', newline='') as myfile:    
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(['Num Samples For Metrics:', metrics_dict['N']])
        wr.writerow(['MDA:', metrics_dict['MDA']])
        wr.writerow(['MAPE:', metrics_dict['MAPE']])
        wr.writerow(['MAE:', metrics_dict['MAE']])
        wr.writerow(['MSE:', metrics_dict['MSE']])
        wr.writerow(['Price Correlation:', metrics_dict['PXCORR']])
        wr.writerow(['Returns Correlation:', metrics_dict['RXCORR']])
        
        
        wr.writerow(['Open Start Price:', start_price])        
        wr.writerow(['Open End Price:', end_price])
        wr.writerow(['Difference (Open to Open) $:', end_price-start_price])
        wr.writerow(['Open Buy and Hold Return %:', 100*(end_price - start_price)/start_price])        
        
        wr.writerow(['Total profits from trading $:', test_profit])        
        wr.writerow(['Return from trading %:', test_return_pct])
        wr.writerow(['Total trades:', num_trades])
        
        wr.writerow(['Total Test Time:', (time.time()-start_time)/60])
        
    # Add the meytrics dict to the result dict
    result_dict.update(metrics_dict)

            
    # return updated model, updated D, Q E and A, journal and results from this test period
    return model, D, Q, E, A, journal, result_dict

In [163]:
def generate_initial_allocation_policy(train_result_dict, percentiles, scale=1, offset=0, basefn='', verbosity=1):
    
    # Varios ways to avoid the initial training period where the model is way off
    stop_idx = len(train_result_dict['val_pred_rt'])
    # Just look at the last 120 time steps - very few data points though
#    start_idx = stop_idx - 120

    # Look at the last 80% of the time steps
    start_idx = stop_idx - int(0.8 * stop_idx)
    
    # Point D at the training predicted returns - for the period we are interested in
    D = np.array(train_result_dict['val_pred_rt'])[start_idx:stop_idx]

    # Compute the percentiles of only the POSITIVE training returns
    # This is what I was doing up to v12, changed in v13
#    Q = np.percentile(D[D >= 0], percentiles)
    
    # Compute the percentiles of the absolute training returns, for the last N time steps
    Q = np.percentile(abs(D), percentiles)
       
    if verbosity >= 1:
        print("\nComputing initial allocation policy from the training set...")
        print("Number of returns from training used for policy:", len(D))
        print("Initial Q:\n",Q)

    # There will be 8 bins from 7 percentiles - anything below the lowest percentile is placed in bin 0
    # Anything equal to or greater than the lowest percentile but less than the 2nd wil be in bin 1 etc
    # Anything equal to or greater than the last percentile will be in the last bin.
    
    if verbosity >= 2:
        binned_returns = np.digitize(D, Q)
        unique, counts = np.unique(binned_returns, return_counts=True)
        print("Unique bins:", unique)
        print("Counts / bin:", counts)

    # Generate the Expected Returns for each bin in Q from the training data
    
    holding = False
    E = np.zeros(shape=(len(percentiles)+1))
    C = np.zeros(shape=(len(percentiles)+1))    
   
    pred_rt = np.array(train_result_dict['val_pred_rt'])
    act_rt = np.array(train_result_dict['val_act_rt'])
    adj_close = np.array(train_result_dict['val_adj_close_price'])
    
    with open(basefn + '_init_alloc.csv', 'w', newline='') as myfile:    
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(['Day','Adj Close','Pred Rt', 'Act Rt','Buy/Sell','Sell Price','Bin','Acc Profit','Total Profit',
                     'E0','E1','E2','E3','E4','E5','E6','E7',
                     'C0','C1','C2','C3','C4','C5','C6','C7',
                     'D0','D1','D2','D3','D4','D5','D6'])
        
    total_profit = 0

    # Walk through the returns in the known valid region of the end of the training data
    for i in range(start_idx, stop_idx):
    
        # This accumulates a line of values for the allocation log file
        init_alloc_results = []
        
        if verbosity >= 2:
            print(i, '{:1.2f}'.format(adj_close[i]), '{:1.4f}'.format(pred_rt[i]))
        
        init_alloc_results=[i,adj_close[i],pred_rt[i], act_rt[i]]
        
        # Sell
        if pred_rt[i] < 0 and holding:
            sell_price = denormalise_value(adj_close[i], scale, offset)
            profit = sell_price - purchase_price
            total_profit += profit
            E[purchase_bin] = E[purchase_bin] + profit
            holding = False
            if verbosity >= 2:
                print("Executed sell at ${:1.2f}".format(sell_price), "on day", i, "Profit: ${:1.4f}".format(profit))
            init_alloc_results += (['Sell', sell_price, purchase_bin, profit, total_profit] + E.tolist() + C.tolist() + Q.tolist())

        # Buy
        if pred_rt[i] >= 0 and not holding:
            return_bin = np.digitize(pred_rt[i], Q)
            purchase_bin = return_bin
            C[purchase_bin] = C[purchase_bin] + 1
            purchase_price = denormalise_value(adj_close[i], scale, offset)
            holding = True
            if verbosity >= 2:
                print("Executed buy at ${:1.2f}".format(purchase_price), "on day ", i, "bin", return_bin)
            init_alloc_results += (['Buy', purchase_price, purchase_bin, 'N/A', total_profit] + E.tolist() + C.tolist() + Q.tolist())
            
        # Append the metrics to the training log file
        with open(basefn + '_init_alloc.csv', 'a', newline='') as myfile:    
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerow(init_alloc_results)

    
    # It's important to understand that bin 0 won't have any trades at this point because we gave it the training
    # data with a certain lower positive return. This value defines the lowest cut off of the training data. However,
    # once we start getting new data we will see values probaby lower than this lowest bin cut off.
    
    A = (E > 0).astype(int)
    
    with open(basefn + '_init_alloc.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow([' ','A0','A1','A2','A3','A4','A5','A6','A7'])
        wr.writerow(['Initial Allocation Policy:'] + A.tolist())
                
    if verbosity >= 1:
        print("Expected Gains per Bin:", E)
        print("Trades per bin:", C)
        print("\nInitial Allocation Policy:",A)        
        
    return D,Q,E,A,C

In [164]:
# resume_dir - the root location of an existing run (including a saved_models directory) if we are picking up from a prevous run
# resume_epoch - the epoch to resume from. Expect to find DQEAC_dictXXX.pkl and epochXXX.mdl files in a saved_models dir where
#                XXX is the epoch number we wish to resume from.  -1 means start from the last.
# train_epochs - how many epochs to train for (if starting from a pre-saved point, how many additonal epochs to train for)

def run_hypers(train_start = '01-01-2005', train_end = '01-01-2008', valid_start = '01-02-2008', valid_end = '12-31-2009',
               test_start = '01-01-2010', test_end = '12-20-2019', 
               resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=1, arch='LH', 
               num_layers=3, hidden_size=128, dropout=0.5, lr=0.01, dr=1.0, wd=0.0, trading_validation_period=100, 
               run_out_of_sample_test=0, train_only=0, verbosity=0, plot=0):

    # If attempting to resume, check resume_dir/saved_models actually exists
    if resume_dir != '':
        if os.path.isdir(resume_dir + '/saved_models') == False:
            print("Error: root directory for resume does not exist!")
            return
    
    hyper_results = []
    
    # Even if we're resuming, we will make a new directory for the new results
    basefn = datetime.today().strftime('%Y-%m-%d-%H-%M-%S' + '/')
    
    # Make a results directory. If the 'directory' doesn't exist, create it
    if os.path.isdir(basefn) == False:
        os.mkdir(basefn)        
                
    hyper_results.append(basefn)
    
    # New additions to log file in CH15 - the root of a resumation and start epoch / how many epochs 
    # If resuming, put the location of the root directory. If starting from scratch put "BASE" in log field
    if resume_dir=='':
        hyper_results.append('BASE')
    else:
        hyper_results.append(resume_dir)
        
    hyper_results.append(resume_epoch)    
    hyper_results.append(train_epochs)
    
    # Go get the data for the asset
    abs_df = get_data(asset_name=asset, directory='../../../data/yahoo_data', start_date='01-01-2005', stop_date='12-20-2019')    
    hyper_results.append(asset)

    hyper_results.append(arch)        
    hyper_results.append(iter_per_seq)
    hyper_results.append(num_layers)        
    hyper_results.append(window_size)        
    hyper_results.append(hidden_size)        
    hyper_results.append(lr)        
    hyper_results.append(dr)
    hyper_results.append(wd)        
    hyper_results.append(dropout)      
        
    plotly_candlestick(abs_df, asset, basefn=basefn, display_plot=plot, save_plot=1)
    
    # Add the Prev Adj Close, drop volume etc
    abs_df = add_features_to_df(abs_df)
    
    # Normalise the entire dataset
    df, scale, offset = normalise_df(abs_df)
    
    plotly_candlestick(df, asset, basefn=basefn, display_plot=plot, save_plot=1)    
                
        
    # Mask off the training data and assign result to a train_asset dataframe
    mask = (df['Date'] >= train_start) & (df['Date'] <= train_end)
    train_asset_df = df.loc[mask]
    
    last_train_date = train_asset_df['Date'].iloc[-1]
    print("last_train_date:", last_train_date)
    
    loc_last_train_date = df.index[df['Date'] == last_train_date].tolist()[0]
    print("loc_last_train_date:", loc_last_train_date)
    
    backup_index = loc_last_train_date - window_size
    print("backup_index:", backup_index)
    
    backup_date = df['Date'].iloc[backup_index]
    print("Backup date:", backup_date)
    
    # Mask off the validation data and assign result to a valid_asset dataframe
#    mask = (df['Date'] >= valid_start) & (df['Date'] <= valid_end)
    mask = (df['Date'] >= backup_date) & (df['Date'] <= valid_end)
    valid_asset_df = df.loc[mask]
    
    # Mask off the test data and assign result to a test_asset dataframe
    mask = (df['Date'] >= test_start) & (df['Date'] <= test_end)
    test_asset_df = df.loc[mask]
        
    # Reset the indices to start from 0 again for each of the dataframes
    train_asset_df.index = np.arange(train_asset_df.shape[0])   
    valid_asset_df.index = np.arange(valid_asset_df.shape[0])    
    test_asset_df.index = np.arange(test_asset_df.shape[0]) 
    
    # Generate validation attribute set
    valid_seq, valid_labels = create_input_sequences_from_df(valid_asset_df, window_size)
    
    print("Train Start Date:", train_asset_df['Date'].iloc[0])
    print("Train End Date:", train_asset_df['Date'].iloc[-1])
    print("Train Length:", len(train_asset_df))
    print("Valid Start Date:", valid_asset_df['Date'].iloc[0])
    print("Valid End Date:", valid_asset_df['Date'].iloc[-1])
    print("Valid Length:", len(valid_asset_df))

    # Convert our list of lists to Tensors
    valid_seq = torch.FloatTensor(valid_seq)
    valid_labels = torch.FloatTensor(valid_labels)
    
        
    percentiles = [0,10,20,30,40,50,60]
#    percentiles = [0,10,20,30,40,50,60,70,80,90]    
       
    model, train_result_dict = create_train_model(train_asset_df, resume_dir=resume_dir, resume_epoch=resume_epoch, 
                                            basefn=basefn, scale=scale, offset=offset, arch=arch, 
                                            window_size = window_size, epochs=train_epochs,
                                            iter_per_seq=iter_per_seq, max_iter=1E32, num_layers=num_layers,
                                            hidden_size=hidden_size, dropout=dropout, lr=lr, dr=dr, wd=wd,
                                            trading_validation_period=trading_validation_period,
                                            percentiles=percentiles,
                                            valid_seq=valid_seq, valid_labels=valid_labels,
                                            verbosity=verbosity, plot=plot)        

    # Append the metrics to the training log file
    with open(basefn + 'train.csv', 'a', newline='') as myfile:    
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(['Num Samples For Metrics:', train_result_dict['trStats_N']])
        wr.writerow(['MDA:', train_result_dict['trMDA']])
        wr.writerow(['MAPE:', train_result_dict['trMAPE']])
        wr.writerow(['MAE:', train_result_dict['trMAE']])
        wr.writerow(['AveTrMSE:', train_result_dict['trAvTrMSE']])        
        wr.writerow(['AveValMSE:', train_result_dict['trAvValMSE']])
        wr.writerow(['Price X-Correl:', train_result_dict['trPXCORR']])        
        wr.writerow(['Returns X-Correl:', train_result_dict['trRXCORR']])    
        
    hyper_results.append(train_result_dict['trAvTrMSE'])
    hyper_results.append(train_result_dict['trAvValMSE'])    
    hyper_results.append(train_result_dict['trMDA'])
    
    if train_only:
        return model, [], train_result_dict
    
    # MODEL TRAINING COMPLETE - NOW WE BUILD THE INITIAL ALLOCATION POLICY
        
    D,Q,E,A,C = generate_initial_allocation_policy(train_result_dict, percentiles, scale=scale, offset=offset, basefn=basefn, verbosity=verbosity)

    policy_dict = {'initial_D':D,'initial_Q':Q,'initial_E':E,'initial_A':A,'initial_C':C}
                   
        
    print("\nPERFORMING VALIDATION...")
        
    model, D, Q, E, A, journal, test_result_dict = test_trading_system(model, valid_seq, valid_labels, D, Q, E, A, percentiles, basefn=basefn+'_val', scale=scale, offset=offset, iter_per_seq=iter_per_seq, lr = 0.001, dr = 1, verbosity=verbosity)

    result_dict = {'val_profit':test_result_dict['trade_profit'], 'val_return_pct':test_result_dict['trade_return']}
        
    hyper_results.append(test_result_dict['MDA'])    
    hyper_results.append(test_result_dict['MSE'])
    hyper_results.append(test_result_dict['num_trades'])    
    hyper_results.append(test_result_dict['PXCORR'])
    hyper_results.append(test_result_dict['RXCORR'])              
    hyper_results.append(test_result_dict['trade_profit'])
    hyper_results.append(test_result_dict['trade_return'])         
    
    policy_dict['post_val_D'] = D
    policy_dict['post_val_Q'] = Q
    policy_dict['post_val_E'] = E
    policy_dict['post_val_A'] = A
                
    if run_out_of_sample_test:
        
        if verbosity >= 1:
            print("\nEVALUATING TEST SET...")    
    
        # Generate test attribute set
        test_seq, test_labels = create_input_sequences_from_df(test_asset_df, window_size)

        # Convert our list of lists to Tensors
        test_seq = torch.FloatTensor(test_seq)
        test_labels = torch.FloatTensor(test_labels)

        model, D, Q, E, A, journal, test_result_dict = test_trading_system(model, test_seq, test_labels, D, Q, E, A, percentiles, basefn=basefn+'test', scale=scale, offset=offset, iter_per_seq=iter_per_seq, lr = 0.001, dr = 1, verbosity=verbosity)
    
        result_dict['test_profit'] = test_result_dict['trade_profit']
        result_dict['test_return_pct'] = test_result_dict['trade_return']
        
        policy_dict['post_test_D'] = D
        policy_dict['post_test_Q'] = Q
        policy_dict['post_test_E'] = E
        policy_dict['post_test_A'] = A
        
    with open('hyper_log.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(hyper_results + policy_dict['initial_E'].tolist() + policy_dict['initial_C'].tolist() + policy_dict['initial_A'].tolist())
        
    with open(basefn + 'policy_dict.pkl','wb') as fp:
        pickle.dump(policy_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)
                    
    return model, journal, result_dict, policy_dict
    


In [ ]:
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2015', train_end = '01-01-2018', valid_start = '01-02-2018', valid_end = '12-18-2019', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [ ]:
model, journal, result, policy_dict = run_hypers(resume_dir='', resume_epoch=-1, window_size=22, asset='SPY', train_epochs=3000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(resume_dir='', resume_epoch=-1, window_size=44, asset='SPY', train_epochs=3000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [ ]:
model, journal, result, policy_dict = run_hypers(resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=3000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=32, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=3000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [ ]:
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2006', train_end = '01-01-2009', valid_start = '01-02-2009', valid_end = '12-31-2010', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [ ]:
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2007', train_end = '01-01-2010', valid_start = '01-02-2010', valid_end = '12-31-2011', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2008', train_end = '01-01-2011', valid_start = '01-02-2011', valid_end = '12-31-2012', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2009', train_end = '01-01-2012', valid_start = '01-02-2012', valid_end = '12-31-2013', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2010', train_end = '01-01-2013', valid_start = '01-02-2013', valid_end = '12-31-2014', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2011', train_end = '01-01-2014', valid_start = '01-02-2014', valid_end = '12-31-2015', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2012', train_end = '01-01-2015', valid_start = '01-02-2015', valid_end = '12-31-2016', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2013', train_end = '01-01-2016', valid_start = '01-02-2016', valid_end = '12-31-2017', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2014', train_end = '01-01-2017', valid_start = '01-02-2017', valid_end = '12-31-2018', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2015', train_end = '01-01-2018', valid_start = '01-02-2018', valid_end = '12-18-2019', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [ ]:
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2012', train_end = '01-01-2015', valid_start = '01-02-2015', valid_end = '12-31-2016', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2013', train_end = '01-01-2016', valid_start = '01-02-2016', valid_end = '12-31-2017', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2014', train_end = '01-01-2017', valid_start = '01-02-2017', valid_end = '12-31-2018', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2015', train_end = '01-01-2018', valid_start = '01-02-2018', valid_end = '12-18-2019', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [ ]:
# This extends the training period while keeping a constant length validation period
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2015', train_end = '01-01-2018', valid_start = '01-02-2018', valid_end = '12-18-2019', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2014', train_end = '01-01-2018', valid_start = '01-02-2018', valid_end = '12-18-2019', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2013', train_end = '01-01-2018', valid_start = '01-02-2018', valid_end = '12-18-2019', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2500, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2012', train_end = '01-01-2018', valid_start = '01-02-2018', valid_end = '12-18-2019', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=3000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2011', train_end = '01-01-2018', valid_start = '01-02-2018', valid_end = '12-18-2019', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=3500, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result, policy_dict = run_hypers(train_start = '01-01-2010', train_end = '01-01-2018', valid_start = '01-02-2018', valid_end = '12-18-2019', resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=4000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)


In [ ]:
model, journal, result, policy_dict = run_hypers(resume_dir='', resume_epoch=-1, window_size=22, asset='SPY', train_epochs=3000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=32, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)


In [ ]:
model, journal, result, policy_dict = run_hypers(resume_dir='', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-3, dr=0.995, wd=0, trading_validation_period=50, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [ ]:
# Example of resuming training on a previous run
model, journal, result, policy_dict = run_hypers(resume_dir='CH14 - Exp 2 - 2000 epochs/2021-03-01-20-54-12', resume_epoch=-1, window_size=11, asset='SPY', train_epochs=2000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=10, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [175]:
def run_out_of_sample_test(model, D, Q, E, A, basefn='', asset='SPY', window_size=11, iter_per_seq=3, lr=1E-3, dr=0):
    
    # Go get the data for the asset
    abs_df = get_data(asset_name=asset, directory='../../../data/yahoo_data', start_date='01-01-2005', stop_date='12-20-2019')

    # Add the Prev Adj Close, drop volume etc
    abs_df = add_features_to_df(abs_df)
    
    # Normalise the entire dataset
    df, scale, offset = normalise_df(abs_df)
    
    # Out of sample test set
    test_start = '01-01-2010'
    test_end = '12-20-2019'
    
#    test_asset_dict = {}
       
#    backup_index = len(test_asset_df) - window_size
#    backup_date = df['Date'].iloc[backup_index]
    
    # Mask off the test data and assign result to a test_asset dataframe
    mask = (df['Date'] >= test_start) & (df['Date'] <= test_end)
    test_asset_df = df.loc[mask]
    
    # Reset the indices to start from 0 again for each of the dataframes
    test_asset_df.index = np.arange(test_asset_df.shape[0]) 
    
    # Generate test attribute set
    test_seq, test_labels = create_input_sequences_from_df(test_asset_df, window_size)

    # Convert our list of lists to Tensors
    test_seq = torch.FloatTensor(test_seq)
    test_labels = torch.FloatTensor(test_labels)
        
    percentiles = [0,10,20,30,40,50,60]

    model, D, Q, E, A, journal, test_result_dict = test_trading_system(model, test_seq, test_labels, D, Q, E, A, percentiles, basefn=basefn, scale=scale, offset=offset, iter_per_seq=iter_per_seq, lr = lr, dr = dr, verbosity=0)
    
    return model, D, Q, E, A, test_result_dict
   
    

In [176]:
def run_trading_validation(model, D, Q, E, A, predictor="pytorch_lstm", drunk_genius_accuracy=0.5, valid_start='01-02-2008', valid_end='12-31-2009', basefn='', asset='SPY', window_size=11, iter_per_seq=3, lr=1E-3, dr=0):
    
    
    # Go get the data for the asset
    abs_df = get_data(asset_name=asset, directory='../../../data/yahoo_data', start_date='01-01-2005', stop_date='12-20-2019')

    # Add the Prev Adj Close, drop volume etc
    abs_df = add_features_to_df(abs_df)
    
    # Normalise the entire dataset
    df, scale, offset = normalise_df(abs_df)
    
    # Training set
#    train_start = '01-01-2005'
#    train_end = '01-01-2008'    
            
    # Mask off the training data and assign result to a train_asset dataframe
#    mask = (df['Date'] >= train_start) & (df['Date'] <= train_end)
#    train_asset_df = df.loc[mask]
    
#    backup_index = len(train_asset_df) - window_size
#    backup_date = df['Date'].iloc[backup_index]
    
    # Mask off the validation data and assign result to a valid_asset dataframe

    mask = (df['Date'] >= valid_start) & (df['Date'] <= valid_end)
    valid_asset_df = df.loc[mask]
           
    # Reset the indices to start from 0 again for each of the dataframes
#    train_asset_df.index = np.arange(train_asset_df.shape[0])   
    valid_asset_df.index = np.arange(valid_asset_df.shape[0])    
    
    print("Valid Start Date:", valid_asset_df['Date'].iloc[0])
    print("Valid End Date:", valid_asset_df['Date'].iloc[-1])
    print("Valid Length:", len(valid_asset_df))

    # Generate validation attribute set
    valid_seq, valid_labels = create_input_sequences_from_df(valid_asset_df, window_size)

    # Convert our list of lists to Tensors
    valid_seq = torch.FloatTensor(valid_seq)
    valid_labels = torch.FloatTensor(valid_labels)
                    
    percentiles = [0,10,20,30,40,50,60]

    model, D, Q, E, A, journal, test_result_dict = test_trading_system(model, valid_seq, valid_labels, D, Q, E, A, percentiles, 
                        predictor=predictor, drunk_genius_accuracy=drunk_genius_accuracy, basefn=basefn, scale=scale, offset=offset, iter_per_seq=iter_per_seq, lr = lr, 
                        dr = dr, verbosity=0)
    
#    print(test_result_dict)
    return model, D, Q, E, A, test_result_dict
   

In [181]:
# This function runs a validation / test repeatability evaluation on a saved model from training

def run_trading_trials(predictor='pytorch_lstm', drunk_genius_accuracy=0.5, valid_start='01-02-2008', valid_end='12-31-2009',savedfn='', model_num=-1, num_trials=10, run_test=0, iter_per_seq=3, lr=1E-3, dr=0):

    model_fn = savedfn + 'epoch' + str(model_num) + '.mdl'
    print(model_fn)
    
    DQEA_fn = savedfn + 'DQEAC_dict' + str(model_num) + '.pkl'
    print(DQEA_fn)
    
    resultsfn = datetime.today().strftime('%Y-%m-%d-%H-%M-%S' + '/')
    
    # Make a results directory. If the 'directory' doesn't exist, create it
    if os.path.isdir(resultsfn) == False:
        os.mkdir(resultsfn)       

    with open(resultsfn + 'val_results.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(['N', 'iter_per_seq', 'lr', 'dr', 'num_trades', 'trade_return'])     

    with open(resultsfn + 'test_results.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(['N', 'num_trades', 'trade_return'])
            
    with open(resultsfn + 'val_test_summary.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(['N','iter_per_seq', 'lr', 'dr', 'mean_val_trades','mean_val_returns','std_val_returns','min_val_returns',
                     'max_val_returns', 'mean_MDA', 'mean_RtCorr', 'mean_MSE', 'mean_MAE', 'mean_MAPE',
                     'mean_test_trades', 'mean_test_returns', 'std_test_returns', 'min_test_returns',
                     'max_test_returns'])
    
    val_trades = []
    test_trades = []
    val_mda = []
    val_mae = []
    val_mape = []
    val_rxcorr = []
    val_mse = []
    val_returns = []
    test_returns = []

    for i in range(num_trials):
        saved_model = torch.load(model_fn)
    
        with open(DQEA_fn, 'rb') as fp:
            saved_DQEAC = pickle.load(fp)
        
        D, Q, E, A, C = saved_DQEAC['D'], saved_DQEAC['Q'], saved_DQEAC['E'], saved_DQEAC['A'], saved_DQEAC['C']    
    
        model, D, Q, E, A, result_dict = run_trading_validation(saved_model, D, Q, E, A, predictor=predictor, 
                                                                drunk_genius_accuracy=drunk_genius_accuracy,
                                                                valid_start=valid_start, valid_end=valid_end, 
                                                                basefn=resultsfn+str(i)+'_', iter_per_seq=iter_per_seq, 
                                                                lr=lr, dr=dr)
        
        print("Val:", i, iter_per_seq, lr,dr, result_dict['num_trades'], result_dict['trade_return'], 
              result_dict['MDA'], result_dict['RXCORR'], result_dict['MSE'])
        
        val_returns.append(result_dict['trade_return'])
        val_trades.append(result_dict['num_trades'])
        val_mda.append(result_dict['MDA'])
        val_mae.append(result_dict['MAE'])        
        val_mape.append(result_dict['MAPE'])                
        val_rxcorr.append(result_dict['RXCORR'])
        val_mse.append(result_dict['MSE'])
        
        with open(resultsfn + 'val_results.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerow([i, iter_per_seq, lr, dr, result_dict['num_trades'],result_dict['trade_return'], result_dict['MDA'],
                        result_dict['RXCORR'], result_dict['MSE'], result_dict['MAE'], result_dict['MAPE']])
    
        if run_test:
            model, D, Q, E, A, test_result_dict = run_out_of_sample_test(model, D, Q, E, A, basefn=resultsfn+str(i)+'_', iter_per_seq=iter_per_seq, lr=lr, dr=dr)
            print("Test:", i, test_result_dict['num_trades'], test_result_dict['trade_return'])
            test_returns.append(test_result_dict['trade_return'])
            test_trades.append(test_result_dict['num_trades'])    
        
            with open(resultsfn + 'test_results.csv', 'a', newline='') as myfile:
                wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                wr.writerow([i,test_result_dict['num_trades'],test_result_dict['trade_return']])

    if run_test:
        print(model_num, np.mean(val_trades),np.mean(val_returns), np.mean(test_trades),np.mean(test_returns))
    else:
        print(model_num, np.mean(val_trades),np.mean(val_returns))
    
    with open(resultsfn + 'val_test_summary.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        if run_test:
            wr.writerow([num_trials,iter_per_seq, lr, dr,np.mean(val_trades),np.mean(val_returns),np.std(val_returns),
                         np.min(val_returns),np.max(val_returns), np.mean(val_mda), np.mean(val_rxcorr), np.mean(val_mse),
                         np.mean(val_mae), np.mean(val_mape), 
                         np.mean(test_trades),np.mean(test_returns),np.std(test_returns),np.min(test_returns),np.max(test_returns)])
        else:
            wr.writerow([i,iter_per_seq, lr, dr,np.mean(val_trades),np.mean(val_returns),np.std(val_returns),np.min(val_returns),
                         np.max(val_returns), np.mean(val_mda), np.mean(val_rxcorr), np.mean(val_mse), np.mean(val_mae), 
                        np.mean(val_mape)])

    

In [184]:
# This is the last 2 years of the rolling period
# savedfn = 'CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/'
# run_trading_trials(valid_start='01-01-2018', valid_end='12-19-2019',savedfn=savedfn, model_num=900, num_trials=10, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)

# This is the best performing period and model I saw in the rolling experiment
# saved_fn = 'CH15 - Exp7 - Roll forward train validation period/2009 - 2021-03-13-14-20-53/saved_models'
#run_trading_trials(valid_start='12-15-2006', valid_end='12-31-2008',savedfn=savedfn, model_num=850, num_trials=10, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)

# 
saved_fn = 'Ch15 - CH16 - Exp8 - Expanding training period/3 year partial - 2021-03-16-13-05-13/saved_models'
run_trading_trials(predictor="pytorch_lstm", valid_start='12-13-2017', valid_end='12-18-2019',savedfn=savedfn, model_num=750, num_trials=25, run_test=0, iter_per_seq=8, lr=1E-4, dr=1)


CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/epoch750.mdl
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/DQEAC_dict750.pkl
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507


C:\Users\ND\Anaconda3\envs\mlp\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2021-03-17-11-10-48/0_
Val: 0 8 0.0001 1 152 -7.133539521489531 46.15384615384615 0.03460598060563963 0.0001487827371172557
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-11-10-48/1_
Val: 1 8 0.0001 1 167 0.7092765891398379 45.74898785425101 0.05953682972960117 0.00014379050859905033
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-11-10-48/2_
Val: 2 8 0.0001 1 151 -9.220177529004477 46.35627530364373 -0.0008317498034556882 0.0001556641817022675
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-11-10-48/3_
Val: 3 8 0.0001 1 138 -6.939473528516862 45.95141700404859 0.003200609224104021 0.00015085230915534034
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-11-10-48/4_
Val: 4 8 0.0001 1 148 -2.165026894285193 45.95141700404859 0.012563977892862338 0.00015019030252363198
Vali

In [182]:
saved_fn = 'Ch15 - CH16 - Exp8 - Expanding training period/3 year partial - 2021-03-16-13-05-13/saved_models'
for drunk_genius_accuracy in np.arange(0,1.1,0.1):
    run_trading_trials(predictor="drunk_genius", drunk_genius_accuracy=drunk_genius_accuracy, valid_start='12-13-2017', valid_end='12-18-2019',savedfn=savedfn, model_num=750, num_trials=25, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)


CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/epoch750.mdl
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/DQEAC_dict750.pkl
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507


C:\Users\ND\Anaconda3\envs\mlp\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2021-03-17-08-37-48/0_
Val: 0 3 0.001 1 247 18.906955076724934 54.65587044534413 -0.012027313183830744 0.00019432880197716397
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-08-37-48/1_
Val: 1 3 0.001 1 218 -9.593433053516762 50.607287449392715 -0.038044944010464536 0.00018804706080234517
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-08-37-48/2_
Val: 2 3 0.001 1 243 10.298968874147286 51.21457489878542 0.011770920479365447 0.00020452177835113166
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-08-37-48/3_
Val: 3 3 0.001 1 232 -9.959280213727768 47.97570850202429 -0.054904206315936524 0.00021113992771034656
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-08-37-48/4_
Val: 4 3 0.001 1 230 11.978797932944037 50.0 -0.02340183748142567 0.000205015511577252
Valid Start Date: 

2021-03-17-08-45-23/12_
Val: 12 3 0.001 1 220 24.969081750957695 55.87044534412956 0.08115633324451639 0.00020651094046361166
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-08-45-23/13_
Val: 13 3 0.001 1 237 15.991650978452899 54.04858299595141 0.06454254983403594 0.00021835112882689618
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-08-45-23/14_
Val: 14 3 0.001 1 236 33.3793940976012 59.311740890688256 0.1775547069115341 0.0002012032220092522
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-08-45-23/15_
Val: 15 3 0.001 1 249 43.420800979967005 59.716599190283404 0.16817927364412608 0.00021602174710860406
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-08-45-23/16_
Val: 16 3 0.001 1 211 2.5010808119695134 53.036437246963565 0.10306382192206022 0.00019611150811611863
Val

2021-03-17-08-53-20/24_
Val: 24 3 0.001 1 218 36.20514972304706 60.526315789473685 0.2542338276025989 0.00022913872219390124
750 236.12 32.98608267792424
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/epoch750.mdl
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/DQEAC_dict750.pkl
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-01-09/0_
Val: 0 3 0.001 1 240 43.71196418221878 64.17004048582996 0.3750972625432654 0.00018204916251974003
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-01-09/1_
Val: 1 3 0.001 1 208 40.70703545176169 63.36032388663968 0.3761610132794404 0.00021415611009434793
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-01-09/2_
Val: 2 3 0.001 1 265 49.2701981383411 61.74089068825911 0.32328993393932315 0.000193707559

2021-03-17-09-09-00/10_
Val: 10 3 0.001 1 256 57.33700191005537 71.25506072874494 0.47537549209828106 0.0001846640234630816
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-09-00/11_
Val: 11 3 0.001 1 244 53.26810304302032 68.62348178137651 0.46648357821384806 0.00020068882540650443
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-09-00/12_
Val: 12 3 0.001 1 251 55.074867353245736 68.62348178137651 0.3701168245961607 0.00017512991244511713
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-09-00/13_
Val: 13 3 0.001 1 248 55.0860254459809 67.61133603238866 0.4408885134277899 0.00021749728559016548
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-09-00/14_
Val: 14 3 0.001 1 235 52.264007602402835 65.58704453441295 0.4614590589735348 0.0001771313927264132
Valid Start

2021-03-17-09-16-37/22_
Val: 22 3 0.001 1 268 76.58366954033202 75.30364372469636 0.5595791979792173 0.0002067364550117651
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-16-37/23_
Val: 23 3 0.001 1 247 75.51611244728325 72.06477732793522 0.4337039673651633 0.0002989888092951921
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-16-37/24_
Val: 24 3 0.001 1 251 55.56759899197948 75.91093117408907 0.5334642307739766 0.00019426006501735259
750 245.68 67.16741965231162
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/epoch750.mdl
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/DQEAC_dict750.pkl
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-24-14/0_
Val: 0 3 0.001 1 229 94.11323631890812 84.21052631578948 0.6635383607927144 0.00020467644

2021-03-17-09-31-50/8_
Val: 8 3 0.001 1 232 88.23030813613956 84.61538461538461 0.7416850221088753 0.00018666965781007685
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-31-50/9_
Val: 9 3 0.001 1 255 88.87606781997991 85.62753036437248 0.7610994857056502 0.00018126012064743446
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-31-50/10_
Val: 10 3 0.001 1 250 82.17552113345302 86.03238866396761 0.6705428601862746 0.00022201002966106722
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-31-50/11_
Val: 11 3 0.001 1 245 97.12574962551643 86.43724696356276 0.765126936769635 0.00022531135272719336
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-31-50/12_
Val: 12 3 0.001 1 247 75.43396039219226 81.57894736842105 0.654971333109397 0.00019173722239949756
Valid Start Date: 

2021-03-17-09-39-23/20_
Val: 20 3 0.001 1 233 103.6918377106147 89.06882591093117 0.8211216049654335 0.0001959261303046462
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-39-23/21_
Val: 21 3 0.001 1 239 106.9207779952889 89.27125506072875 0.8387391333416466 0.00020848000294192278
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-39-23/22_
Val: 22 3 0.001 1 241 86.68483911960114 87.4493927125506 0.7446549827577189 0.0002262607383759258
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-39-23/23_
Val: 23 3 0.001 1 241 103.038600969718 88.46153846153847 0.8455458303322488 0.00017781692818234654
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-39-23/24_
Val: 24 3 0.001 1 251 106.60352647932146 89.67611336032388 0.9009192383248381 0.00021091821580886122
750 239.52 101.

2021-03-17-09-54-28/7_
Val: 7 3 0.001 1 231 125.62992183056957 99.59514170040485 1.0 0.00017645553390966884
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-54-28/8_
Val: 8 3 0.001 1 231 125.62992183056957 99.59514170040485 1.0 0.00019526949133782484
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-54-28/9_
Val: 9 3 0.001 1 231 125.62992183056957 99.59514170040485 1.0 0.00018007028907294912
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-54-28/10_
Val: 10 3 0.001 1 231 125.62992183056957 99.59514170040485 1.0 0.00018817647228185116
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 507
2021-03-17-09-54-28/11_
Val: 11 3 0.001 1 231 125.62992183056957 99.59514170040485 1.0 0.00021583165486500713
Valid Start Date: 2017-12-13 00:00:00
Valid End Date: 2019-12-18 00:00:00
Valid Length: 

In [187]:
# 2008
saved_fn = 'CH15 - Exp7 - Roll forward train validation period/2008 - 2021-03-03-17-09-15 LR1E-4 LRD0 - baseline from CH15 Exp2/saved_models'
run_trading_trials(predictor="pytorch_lstm", valid_start='12-14-2007', valid_end='12-31-2009',savedfn=savedfn, model_num=1050, num_trials=25, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)


CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/epoch1050.mdl
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/DQEAC_dict1050.pkl
Valid Start Date: 2007-12-14 00:00:00
Valid End Date: 2009-12-31 00:00:00
Valid Length: 516


C:\Users\ND\Anaconda3\envs\mlp\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2021-03-17-18-02-07/0_
Val: 0 3 0.001 1 73 -18.694769470467115 51.888667992047715 -0.006991849304677796 0.0043355595822443934
Valid Start Date: 2007-12-14 00:00:00
Valid End Date: 2009-12-31 00:00:00
Valid Length: 516
2021-03-17-18-02-07/1_
Val: 1 3 0.001 1 108 9.29975998748533 54.27435387673956 0.0029968163903502845 0.0036985772455767514
Valid Start Date: 2007-12-14 00:00:00
Valid End Date: 2009-12-31 00:00:00
Valid Length: 516
2021-03-17-18-02-07/2_
Val: 2 3 0.001 1 76 -7.647854057124448 50.89463220675945 0.018644999197724204 0.0032595657856788578
Valid Start Date: 2007-12-14 00:00:00
Valid End Date: 2009-12-31 00:00:00
Valid Length: 516
2021-03-17-18-02-07/3_
Val: 3 3 0.001 1 93 -14.269216941748885 49.90059642147117 -0.000224466943508298 0.004915943413500126
Valid Start Date: 2007-12-14 00:00:00
Valid End Date: 2009-12-31 00:00:00
Valid Length: 516
2021-03-17-18-02-07/4_
Val: 4 3 0.001 1 92 -26.45141909826697 46.71968190854871 -0.035545937447736736 0.002924730723409767
Valid Start D

In [188]:
# 2009
saved_fn = 'CH15 - Exp7 - Roll forward train validation period/2009 - 2021-03-13-14-20-53/saved_models'
run_trading_trials(predictor="pytorch_lstm", valid_start='12-13-2008', valid_end='12-31-2010',savedfn=savedfn, model_num=850, num_trials=25, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)

# 2010
saved_fn = 'CH15 - Exp7 - Roll forward train validation period/2010 - 2021-03-13-21-37-25/saved_models'
run_trading_trials(predictor="pytorch_lstm", valid_start='12-13-2009', valid_end='12-31-2011',savedfn=savedfn, model_num=650, num_trials=25, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)

# 2011
saved_fn = 'CH15 - Exp7 - Roll forward train validation period/2011 - 2021-03-14-04-34-53/saved_models'
run_trading_trials(predictor="pytorch_lstm", valid_start='12-13-2010', valid_end='12-31-2012',savedfn=savedfn, model_num=700, num_trials=25, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)

# 2012
saved_fn = 'CH15 - Exp7 - Roll forward train validation period/2012 - 2021-03-14-11-33-41/saved_models'
run_trading_trials(predictor="pytorch_lstm", valid_start='12-13-2011', valid_end='12-31-2013',savedfn=savedfn, model_num=450, num_trials=25, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)

# 2013
saved_fn = 'CH15 - Exp7 - Roll forward train validation period/2013 - 2021-03-14-18-33-45/saved_models'
run_trading_trials(predictor="pytorch_lstm", valid_start='12-13-2012', valid_end='12-31-2014',savedfn=savedfn, model_num=650, num_trials=25, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)

# 2014
saved_fn = 'CH15 - Exp7 - Roll forward train validation period/2014 - 2021-03-15-01-30-56/saved_models'
run_trading_trials(predictor="pytorch_lstm", valid_start='12-13-2013', valid_end='12-31-2015',savedfn=savedfn, model_num=1050, num_trials=25, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)

# 2015
saved_fn = 'CH15 - Exp7 - Roll forward train validation period/2015 - 2021-03-15-08-29-15/saved_models'
run_trading_trials(predictor="pytorch_lstm", valid_start='12-13-2014', valid_end='12-31-2016',savedfn=savedfn, model_num=650, num_trials=25, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)

# 2016
saved_fn = 'CH15 - Exp7 - Roll forward train validation period/2016 - 2021-03-15-13-41-40/saved_models'
run_trading_trials(predictor="pytorch_lstm", valid_start='12-13-2015', valid_end='12-31-2017',savedfn=savedfn, model_num=900, num_trials=25, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)

# 2017
saved_fn = 'CH15 - Exp7 - Roll forward train validation period/2017 - 2021-03-15-18-54-04/saved_models'
run_trading_trials(predictor="pytorch_lstm", valid_start='12-13-2016', valid_end='12-31-2018',savedfn=savedfn, model_num=300, num_trials=25, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)

# 2018
saved_fn = 'CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models'
run_trading_trials(predictor="pytorch_lstm", valid_start='12-13-2017', valid_end='12-31-2019',savedfn=savedfn, model_num=900, num_trials=25, run_test=0, iter_per_seq=3, lr=1E-3, dr=1)



CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/epoch850.mdl
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/DQEAC_dict850.pkl
Valid Start Date: 2008-12-15 00:00:00
Valid End Date: 2010-12-31 00:00:00
Valid Length: 516


C:\Users\ND\Anaconda3\envs\mlp\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2021-03-17-18-21-03/0_
Val: 0 3 0.001 1 77 12.9585054771556 46.3220675944334 0.07596194857855629 0.005800155911935785
Valid Start Date: 2008-12-15 00:00:00
Valid End Date: 2010-12-31 00:00:00
Valid Length: 516
2021-03-17-18-21-03/1_
Val: 1 3 0.001 1 86 30.499374501096646 53.28031809145129 0.06738764633190543 0.0035737333870778745
Valid Start Date: 2008-12-15 00:00:00
Valid End Date: 2010-12-31 00:00:00
Valid Length: 516
2021-03-17-18-21-03/2_
Val: 2 3 0.001 1 93 14.066971791465665 48.508946322067594 0.08917500249795342 0.004439595097556912
Valid Start Date: 2008-12-15 00:00:00
Valid End Date: 2010-12-31 00:00:00
Valid Length: 516
2021-03-17-18-21-03/3_
Val: 3 3 0.001 1 42 -9.633207105656567 45.32803180914513 0.06421463019691313 0.0069895100811285015
Valid Start Date: 2008-12-15 00:00:00
Valid End Date: 2010-12-31 00:00:00
Valid Length: 516
2021-03-17-18-21-03/4_
Val: 4 3 0.001 1 72 26.36272982295253 51.491053677932406 0.054551921071041225 0.00718673135270485
Valid Start Date: 2008-12-1

Val: 12 3 0.001 1 119 15.0252228970387 53.37301587301587 0.02028764379704021 0.009170862818661253
Valid Start Date: 2009-12-14 00:00:00
Valid End Date: 2011-12-30 00:00:00
Valid Length: 517
2021-03-17-18-28-31/13_
Val: 13 3 0.001 1 95 19.246117594146174 52.57936507936508 0.016177727276095268 0.007814309222277195
Valid Start Date: 2009-12-14 00:00:00
Valid End Date: 2011-12-30 00:00:00
Valid Length: 517
2021-03-17-18-28-31/14_
Val: 14 3 0.001 1 87 -12.618346664679336 48.01587301587302 -0.0004410918628924303 0.010200755227692948
Valid Start Date: 2009-12-14 00:00:00
Valid End Date: 2011-12-30 00:00:00
Valid Length: 517
2021-03-17-18-28-31/15_
Val: 15 3 0.001 1 97 8.052430680131133 48.41269841269841 0.0242897331648615 0.009355385775695862
Valid Start Date: 2009-12-14 00:00:00
Valid End Date: 2011-12-30 00:00:00
Valid Length: 517
2021-03-17-18-28-31/16_
Val: 16 3 0.001 1 95 -12.503320689322813 53.17460317460318 0.00721448136441386 0.009327812246762972
Valid Start Date: 2009-12-14 00:00:00


2021-03-17-18-36-02/24_
Val: 24 3 0.001 1 104 -6.664008499927466 45.52683896620278 0.01342805261481058 0.0043335447642798884
700 91.48 3.887846954460636
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/epoch450.mdl
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/DQEAC_dict450.pkl
Valid Start Date: 2011-12-13 00:00:00
Valid End Date: 2013-12-31 00:00:00
Valid Length: 515
2021-03-17-18-44-10/0_
Val: 0 3 0.001 1 91 30.44028627562901 50.398406374501995 0.034594999276304404 0.008837043515170676
Valid Start Date: 2011-12-13 00:00:00
Valid End Date: 2013-12-31 00:00:00
Valid Length: 515
2021-03-17-18-44-10/1_
Val: 1 3 0.001 1 84 31.01735896677717 50.0 0.030148826994522173 0.00898281383522813
Valid Start Date: 2011-12-13 00:00:00
Valid End Date: 2013-12-31 00:00:00
Valid Length: 515
2021-03-17-18-44-10/2_
Val: 2 3 0.001 1 97 30.076598766081712 51.59362549800797 0.025936307756561937 0.010267921685706276
Valid S

2021-03-17-18-51-54/11_
Val: 11 3 0.001 1 127 2.4175231845201774 44.33399602385686 0.09951793817391383 0.00520011887498713
Valid Start Date: 2012-12-13 00:00:00
Valid End Date: 2014-12-31 00:00:00
Valid Length: 516
2021-03-17-18-51-54/12_
Val: 12 3 0.001 1 133 9.45685430772892 46.3220675944334 0.10322046052079437 0.006762258913519215
Valid Start Date: 2012-12-13 00:00:00
Valid End Date: 2014-12-31 00:00:00
Valid Length: 516
2021-03-17-18-51-54/13_
Val: 13 3 0.001 1 133 6.178883487608975 45.52683896620278 0.10513365103063647 0.00764923793845551
Valid Start Date: 2012-12-13 00:00:00
Valid End Date: 2014-12-31 00:00:00
Valid Length: 516
2021-03-17-18-51-54/14_
Val: 14 3 0.001 1 113 7.84279964575784 46.71968190854871 0.13854505163607658 0.004297954960858249
Valid Start Date: 2012-12-13 00:00:00
Valid End Date: 2014-12-31 00:00:00
Valid Length: 516
2021-03-17-18-51-54/15_
Val: 15 3 0.001 1 133 9.094192583130212 43.538767395626245 0.11418835607759162 0.004204869562624919
Valid Start Date: 20

2021-03-17-19-00-07/23_
Val: 23 3 0.001 1 142 3.6547554119212293 49.10536779324055 -0.0561762096582935 0.00016555158226849593
Valid Start Date: 2013-12-13 00:00:00
Valid End Date: 2015-12-31 00:00:00
Valid Length: 516
2021-03-17-19-00-07/24_
Val: 24 3 0.001 1 128 -4.345442184209056 47.11729622266402 -0.05176313204689035 8.489255757443771e-05
1050 155.88 10.28620200126479
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/epoch650.mdl
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/DQEAC_dict650.pkl
Valid Start Date: 2014-12-15 00:00:00
Valid End Date: 2016-12-30 00:00:00
Valid Length: 516
2021-03-17-19-08-18/0_
Val: 0 3 0.001 1 162 8.083221738910066 54.07554671968191 0.13916981268018877 8.202280836257311e-05
Valid Start Date: 2014-12-15 00:00:00
Valid End Date: 2016-12-30 00:00:00
Valid Length: 516
2021-03-17-19-08-18/1_
Val: 1 3 0.001 1 152 3.261136090938887 51.888667992047715 0.14908937658809718 7.0137

2021-03-17-19-16-53/9_
Val: 9 3 0.001 1 187 29.620898781210688 55.06958250497018 0.27239485723035645 3.838205108637282e-05
Valid Start Date: 2015-12-14 00:00:00
Valid End Date: 2017-12-29 00:00:00
Valid Length: 516
2021-03-17-19-16-53/10_
Val: 10 3 0.001 1 192 28.323153708797346 50.49701789264414 0.2918050493208907 4.1343198183656335e-05
Valid Start Date: 2015-12-14 00:00:00
Valid End Date: 2017-12-29 00:00:00
Valid Length: 516
2021-03-17-19-16-53/11_
Val: 11 3 0.001 1 190 29.325496690087896 54.67196819085487 0.24087599556290964 4.263863849127081e-05
Valid Start Date: 2015-12-14 00:00:00
Valid End Date: 2017-12-29 00:00:00
Valid Length: 516
2021-03-17-19-16-53/12_
Val: 12 3 0.001 1 188 28.65236138692989 54.87077534791253 0.30558698741526147 3.997468020734312e-05
Valid Start Date: 2015-12-14 00:00:00
Valid End Date: 2017-12-29 00:00:00
Valid Length: 516
2021-03-17-19-16-53/13_
Val: 13 3 0.001 1 188 30.313300214586235 55.06958250497018 0.3039714271718403 3.585172996876595e-05
Valid Start

2021-03-17-19-26-15/21_
Val: 21 3 0.001 1 168 13.086434915572422 51.99203187250996 0.04108792943181974 0.00012479046488808124
Valid Start Date: 2016-12-13 00:00:00
Valid End Date: 2018-12-31 00:00:00
Valid Length: 515
2021-03-17-19-26-15/22_
Val: 22 3 0.001 1 178 -3.8354366646083884 49.601593625498005 -0.021858656720119572 0.00013104147220747284
Valid Start Date: 2016-12-13 00:00:00
Valid End Date: 2018-12-31 00:00:00
Valid Length: 515
2021-03-17-19-26-15/23_
Val: 23 3 0.001 1 170 -3.305862101765168 51.79282868525896 0.04712071860232758 0.00011954888625318414
Valid Start Date: 2016-12-13 00:00:00
Valid End Date: 2018-12-31 00:00:00
Valid Length: 515
2021-03-17-19-26-15/24_
Val: 24 3 0.001 1 164 7.09714842481421 52.58964143426295 -0.016184398675422925 0.00013066263542980822
300 166.64 2.909824656705365
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved_models/epoch900.mdl
CH15 - Exp7 - Roll forward train validation period/2018 - 2021-03-16-00-08-34/saved

In [ ]:
model, journal, result, policy_dict = run_hypers(window_size=11, asset='SPY', train_epochs=4000, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=10, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [ ]:
import pickle
with open('policy_dict.pkl','wb') as fp:
    pickle.dump(policy_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)
    


In [ ]:
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=10, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=10, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)


In [ ]:
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, trading_validation_period=10, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=2, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=4, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=5, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=6, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.5, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.99, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=1)

In [ ]:
for trial in range(100):
    iter_per_seq = random.choice([10, 20, 30, 40, 50, 60 ,70, 80, 90, 100])
    window_size = random.choice([11, 22, 44])
    hidden_size = random.choice([32, 64, 128])
    num_layers = random.choice([2, 3])
    arch = random.choice(['LH','AH'])
    dropout = random.choice([0, 0.5, 0.7])
    lr = random.choice([1E-3, 5E-4, 1E-4, 5E-5, 1E-5])
    dr = random.choice([0.9, 0.93, 0.95, 0.97, 0.99, 1])
    model, journal, result = run_hypers(window_size=window_size, asset='SPY', train_epochs=1, iter_per_seq=iter_per_seq, arch=arch, num_layers=num_layers, hidden_size=hidden_size, dropout=dropout, lr=lr, dr=dr, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    
    

In [ ]:
for r in range(10):
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=40, arch='AH', num_layers=2, hidden_size=64, dropout=0.7, lr=1E-4, dr=0.95, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    

In [ ]:
for r in range(10):
    model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=2, hidden_size=32, dropout=0.7, lr=1E-3, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    

In [ ]:
model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=2, hidden_size=32, dropout=0, lr=1E-3, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)
model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=2, hidden_size=32, dropout=0.1, lr=1E-3, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    
model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=2, hidden_size=32, dropout=0.2, lr=1E-3, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    
model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=2, hidden_size=32, dropout=0.3, lr=1E-3, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    
model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=2, hidden_size=32, dropout=0.4, lr=1E-3, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    
model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=2, hidden_size=32, dropout=0.5, lr=1E-3, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    
model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=2, hidden_size=32, dropout=0.6, lr=1E-3, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    
model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=2, hidden_size=32, dropout=0.7, lr=1E-3, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    
model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=2, hidden_size=32, dropout=0.8, lr=1E-3, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    
model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=2, hidden_size=32, dropout=0.9, lr=1E-3, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    
model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=2, hidden_size=32, dropout=1, lr=1E-3, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=1, plot=0)    

In [ ]:
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=2, plot=1)    

In [ ]:
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=2, plot=1)    

In [ ]:
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=2, plot=1)    

In [ ]:
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=2, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=2, plot=1)    

In [ ]:
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=3, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=2, plot=1)    

In [ ]:
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=4, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=2, plot=1)    

In [ ]:
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=5, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=2, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1500, iter_per_seq=6, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=1E-4, dr=1, wd=0, run_out_of_sample_test=0, train_only=0, verbosity=2, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.000001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.000001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.000001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.000001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.000001, dr=1, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.99, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.99, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.99, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.99, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.99, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.9, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.9, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.9, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.9, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.9, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.90, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.91, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.92, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.93, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.94, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.96, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.97, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.98, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.99, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.90, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.91, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.92, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.93, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.94, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.96, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.97, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.98, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.99, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.00001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.000001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.000001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.000001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.000001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.000001, dr=0.95, wd=0, run_out_of_sample_test=0, verbosity=1, plot=1)

In [ ]:
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=0.999, wd=0, run_out_of_sample_test=0, verbosity=2, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=0.999, wd=1E-8, run_out_of_sample_test=0, verbosity=2, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=0.999, wd=1E-7, run_out_of_sample_test=0, verbosity=2, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=0.999, wd=1E-6, run_out_of_sample_test=0, verbosity=2, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=0.999, wd=1E-5, run_out_of_sample_test=0, verbosity=2, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=0.999, wd=1E-4, run_out_of_sample_test=0, verbosity=2, plot=1)
model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=0.999, wd=1E-3, run_out_of_sample_test=0, verbosity=2, plot=1)


In [ ]:
# r1 - r8 run on 16/2/21-18/2/21
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=1, wd=5E-6, run_out_of_sample_test=0, verbosity=2, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, wd=5E-6, run_out_of_sample_test=0, verbosity=2, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, wd=5E-6, run_out_of_sample_test=0, verbosity=2, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, wd=5E-6, run_out_of_sample_test=0, verbosity=2, plot=1)
    model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, wd=5E-6, run_out_of_sample_test=0, verbosity=2, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.0001, dr=1, wd=5E-6, run_out_of_sample_test=0, verbosity=2, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.0001, dr=1, wd=5E-6, run_out_of_sample_test=0, verbosity=2, plot=1)
    model, journal, result = run_hypers(window_size=11, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.0001, dr=1, wd=5E-6, run_out_of_sample_test=0, verbosity=2, plot=1)

In [ ]:
# make_synthetic_stock(source_asset_name='SPY', new_name='SYN', directory='../../../data/yahoo_data', start_date='01-01-2005', stop_date='12-20-2019', trend_list=[(10,37),(7,17),(3,7)], adj_close_start=85, price_slope=0, close_scale=1)

In [ ]:
make_synthetic_stock(source_asset_name='SPY', new_name='SYN2', directory='../../../data/yahoo_data', start_date='01-01-2005', stop_date='12-20-2019', trend_list=[(10,33),(5,17)], adj_close_start=85, price_slope=0, close_scale=1)

In [ ]:
make_synthetic_stock(source_asset_name='SPY', new_name='SYN3', directory='../../../data/yahoo_data', start_date='01-01-2005', stop_date='12-20-2019', trend_list=[(10,33),(5,17)], adj_close_start=85, price_slope=0.01, close_scale=1)

In [ ]:
make_synthetic_stock(source_asset_name='SPY', new_name='SYN4', directory='../../../data/yahoo_data', start_date='01-01-2005', stop_date='12-20-2019', trend_list=[(10,33),(5,17),(3,11),(2,3)], adj_close_start=85, price_slope=0.01, close_scale=1)

In [ ]:
make_synthetic_stock(source_asset_name='SPY', new_name='SYN4S', directory='../../../data/yahoo_data', start_date='01-01-2005', stop_date='03-31-2005', trend_list=[(10,33),(5,17),(3,11),(2,3)], adj_close_start=85, price_slope=0.01, close_scale=1)

In [ ]:
iterations_win_leg()
epochs_win_leg()
iterations_lr_leg()



In [ ]:
# SPY Num Iterations Leg - using optimum settings from SYN4 experiments
def spy_iterations_leg():
    %mkdir "num_iter"
    %cd num_iter
    
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=50, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=200, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=300, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=400, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    %cd ..
    

In [ ]:
# SPY Num Iterations Leg - using optimum settings from SYN4 experiments
def spy_iterations_leg2():
    %mkdir "num_iter2"
    %cd num_iter
    
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1100, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1200, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1300, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1400, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=1500, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
#    model, journal, result = run_hypers(window_size=44, asset='SPY', train_epochs=1, iter_per_seq=400, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    %cd ..

In [ ]:
spy_iterations_leg2()

In [ ]:
# Num Iterations Leg - using optimum settings from SYN4 experiments
def iterations_leg():
    %mkdir "num_iter"
    %cd num_iter
    
    model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=5, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=20, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=40, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=80, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    %cd ..
    

In [ ]:
iterations_leg()

In [ ]:
# Iterations, Window size leg
def iterations_win_leg():
    %mkdir "iter_win_len"
    %cd iter_win_len
    model, journal, result = run_hypers(window_size=1, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=2, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=3, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=7, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=11, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=88, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    %cd ..


In [ ]:
# Epochs, Window size leg
def epochs_win_leg():
    %mkdir "epoch_win_len"
    %cd epoch_win_len

    model, journal, result = run_hypers(window_size=1, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=2, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=3, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=7, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=11, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=44, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=88, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)

    %cd ..

In [ ]:
# Iterations, Learning rate leg
def iterations_lr_leg():
    %mkdir "iter_lr"
    %cd iter_lr
    
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.000001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.00001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.01, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.1, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)

    %cd ..

In [ ]:
# Epoch, Learning rate leg
def epoch_lr_leg():
    %mkdir "epoch_lr"
    %cd epoch_lr
    
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.000001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.00001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.0001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.01, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.1, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    
    %cd ..


In [ ]:
# Iterations, Hidden size leg
def iterations_hidden_size_leg():
    %mkdir "iter_hidden_len"
    %cd iter_hidden_len
    
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=2, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=4, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=8, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=16, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=32, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=1, iter_per_seq=10, arch='LH', num_layers=3, hidden_size=256, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    
    %cd ..

In [ ]:
# Epochs, Hidden size leg
def epochs_hidden_size_leg():
    %mkdir "epoch_hidden_len"
    %cd epoch_hidden_len
    
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=2, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=4, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=8, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=16, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=32, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=64, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=128, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    model, journal, result = run_hypers(window_size=22, asset='SYN4', train_epochs=10, iter_per_seq=1, arch='LH', num_layers=3, hidden_size=256, dropout=0.5, lr=0.001, dr=1, run_out_of_sample_test=0, verbosity=1, plot=1)
    
    %cd ..

In [ ]:
# Everything past here is graveyard / experimental

In [ ]:
# THIS IS THE OLD (SLOW) VERSION OF TRAIN_MODEL - IT WAS REPLACED BY TRAIN_MODELQ

# Trains the given LSTM model. Also collects validation data (predicting the next day) and collects stats
# for both.  These are used to observe training vs validation but also to construct the first allocation policy.

def train_model(model, train_seq, train_labels, scale=1, offset=1, train_start=0, train_length=1, epochs = 20, iter_per_seq=1, max_iter=1600, lr = 0.001, dr=0.999, wd=0.0, verbosity=0, basefn='',percentiles=[0,10,20,30,40,50,60], trading_validation_period=100, valid_seq=[], valid_labels=[], plot=0):
    input_size = 6
    
    
    # if plot level is 4, plot graphs on screen for every inner loop training run
    if plot == 4:
        dump_log_interval = 1
    else:
        dump_log_interval = 10 # otherwise do it every 10
        
    loss_function = nn.MSELoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay = wd)
    my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer = optimizer, gamma = dr)
    
    batch_size = 1  # batch mode isn't used actually but including for future possible use
    
    with open(basefn + 'train.csv', 'a', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(['Epc','AvTrLoss','AvTrMSE','AvValMSE','MdnTrMSE','MdnValMSE','ValMDA','ValMAPE','ValMAE','PXcorr','RXcorr',
                     'EpcSecs','TtlMins'])
        
    with open(basefn + 'trade_validation.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(['Epoch','MDA','MSE','MAE','MAPE','X-Correl','Num-trades','AveRt%','MdnRT%','MinRt%','MaxRt%',
                    'RangeRt%','StdDevRt%'])
            
    # Outputs a summary line for each Epoch
    if verbosity == 1:
        print('Epc  AvTrLoss  AvTrMSE   AvValMSE  MdnTrMSE  MdnValMSE  ValMDA    ValMAPE  ValMAE    PXcorr  RXcorr Secs    TTlMins')

    # Outputs a summary line for each sequence in each epoch
    if verbosity == 2:
        print('EPc  Seq#   TrLoss    TrMSE     PredR   ActRt   ValLoss   VAdjC    VPred   VAct      VPredRt   VActRt   RunMDA')            
                            
    
#    train_losses = []
#    val_adj_close_price = []
    ave_grads = []
    max_grads = []
    min_grads = []
    
    # If the number of epochs is high, just sample the gradients stochastically to avoid retaining too much
    # data.
    if (epochs * train_length) > 1000:
        sample_grads = 1000.0 / (epochs * train_length)
    else:
        sample_grads = 1.0

    start_time = time.time()        

    # How many times through the entire training set
    for i in range(epochs):
        train_losses = []
        val_adj_close_price = []        
        train_mses = []
        val_mses = []
        val_pred_rt = []
        val_act_rt = []
        val_pred_price = []
        val_act_price = []        
        running_mda = 0
        epoch_start_time = time.time()
        # Each sample in the training set
        for j in range(train_start, train_start + train_length):      

            # Reset the learning rate each time for the inner loop
            optimizer.param_groups[0]['lr'] = lr
            
            t_mse_list = []
            v_mse_list = []
            il_ave_grads = []
            il_max_grads = []
            
#            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay = wd)
#            my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer = optimizer, gamma = dr)
            
#            if j%dump_log_interval == 0:
#                dump_log = 1
#            else:
            # Disable inner loop instrumentations for now
            dump_log = 0
            
            # How many times we train on each sample as we go
            for iter in range(iter_per_seq):
                # Send input / label data to Cuda if available
                inputs = train_seq[j].to(device)
                labels = train_labels[j].to(device)
        
                model.train()
            
                # Initialise hidden states before every training event
                h = model.init_hidden(batch_size)  
                                    
                optimizer.zero_grad()
        
                y_pred, h, lstm_out = model(inputs, h)
            
                if dump_log:
                    tp = y_pred[-1].item()
                    ta = labels[-1].item()
                    t_mse = (tp-ta)**2

                # y_pred and labels are both size T - sequence to sequence loss function
                train_loss = loss_function(y_pred, labels)
            
                # Compute gradients and update the model
                train_loss.backward()
                                
                optimizer.step()

                my_lr_scheduler.step()
                
                # Do an inner loop validation
                if dump_log:
                    inputs = train_seq[j+1].to(device)
                    labels = train_labels[j+1].to(device)
            
                    with torch.no_grad():
                        h = model.init_hidden(batch_size)
                        y_pred, h, lstm_out = model(inputs, h)
                
                        vp = y_pred[-1].item()
                        va = labels[-1].item()
                        v_mse = (vp-va)**2
                
                    # write detailed training log
                    with open(basefn + 'train_inner.csv', 'a', newline='') as myfile:        
                        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                        wr.writerow([j,iter,train_loss.item(),tp,ta,t_mse,vp,va,v_mse])
                    
                    t_mse_list.append(t_mse)
                    v_mse_list.append(v_mse)

                    # Grab the inner loop grads - only if we're plotting as we go
                    if plot >= 3:                    
                        il_ave_grad = []
                        il_max_grad = []
                        for n,p in model.named_parameters():
                            if (p.requires_grad) and ("bias" not in n):
                                il_ave_grad.append(p.grad.abs().mean())
                                il_max_grad.append(p.grad.abs().max())
                        il_ave_grads.append(il_ave_grad)
                        il_max_grads.append(il_max_grad)
            
            # After the inner loop completes, plot the inner-loop charts if enabled
            if dump_log and plot >= 3:
                plot_losses(t_mse_list, v_mse_list, title='Train / Validation Loss. Seq:'+str(j), basefn=basefn, display_plot=1, save_plot=0)
                print("last tp:",tp,"\tlast ta:",ta,"\tlast t_mse:",t_mse)
                print("last vp:",vp,"\tlast va:",va,"\tlast v_mse:",v_mse)
            
                plot_gradients(model, il_ave_grads, il_max_grads, basefn=basefn, display_plot=1, save_plot=0)

            # Sample stochastically the last set of gradients after the inner loop for plotting later
            if plot and (random.random() < sample_grads):
                ave_grad=[]
                max_grad=[]
                min_grad=[]
                for n,p in model.named_parameters():
                    if (p.requires_grad) and ("bias" not in n):
                        ave_grad.append(p.grad.abs().mean())
                        max_grad.append(p.grad.abs().max())
                        min_grad.append(p.grad.abs().min())
                ave_grads.append(ave_grad)
                max_grads.append(max_grad)
                min_grads.append(min_grad)            
            
            # Capture the (snapshot) metrics from the last inner loop cycle
            training_loss = train_loss.item()  # This is the sequence to sequence training loss                        
            train_adj_close = inputs[-1][-1].item()   # Adj close at t=0
            train_pred = y_pred[-1].item()  # Next day predicted Adj close
            train_truth = labels[-1].item()  # Next day actual Adj close
            train_mse = (train_pred - train_truth)**2   # MSE of just the predicted value vs truth
            train_predicted_return = (denormalise_value(train_pred, scale, offset) / denormalise_value(train_adj_close, scale, offset)) - 1            
            train_actual_return = (denormalise_value(train_truth, scale, offset) / denormalise_value(train_adj_close, scale, offset)) - 1

            # Outer loop validation at the end of inner loop training - we always do this regardless of what
            # validation we did in the inner loop (which is usually sampled)
            model.eval()
            
            inputs = train_seq[j+1].to(device)
            labels = train_labels[j+1].to(device)
            
            with torch.no_grad():
                h = model.init_hidden(batch_size)
        
                y_pred, h, lstm_out = model(inputs, h)
                    
                valid_adj_close = inputs[-1][-1].item()                
                valid_pred = y_pred[-1].item()
                valid_truth = labels[-1].item()
                valid_loss = (valid_pred - valid_truth)**2  # Just the mse of the valid predicted value vs truth
            
                valid_predicted_return = (denormalise_value(valid_pred, scale, offset) / denormalise_value(valid_adj_close, scale, offset)) - 1
                valid_actual_return = (denormalise_value(valid_truth, scale, offset) / denormalise_value(valid_adj_close, scale, offset)) - 1

                if (valid_predicted_return * valid_actual_return >= 0):
                    running_mda += 1                
                
                # If enabled, print results line at the end of inner loop training on each sequence
                if (verbosity==2):
                    print('{:4}'.format(i),
                            '{:4}'.format(j), 
                          '  {:1.1e}'.format(training_loss),
                          '  {:1.1e}'.format(train_mse),
                          ' {: 2.3f}'.format(100.0*train_predicted_return), 
                          ' {: 2.3f}'.format(100.0*train_actual_return),
                          '  {:1.1e}'.format(valid_loss), 
                          ' {: 1.4f}'.format(valid_adj_close), 
                          ' {: 1.4f}'.format(valid_pred), 
                          ' {: 1.4f}'.format(valid_truth),
                          ' {: 2.4f}'.format(100.0*valid_predicted_return), 
                          ' {: 2.4f}'.format(100.0*valid_actual_return),
                          '  {:2.2f}'.format(100.0*running_mda/(j+1)))
                    
                # Keep lists of the results of train / validation for each sequence for passing back up the stack
                train_losses.append(training_loss)
                train_mses.append(train_mse)
                val_mses.append(valid_loss)
                val_pred_rt.append(valid_predicted_return)
                val_act_rt.append(valid_actual_return)
                val_pred_price.append(valid_pred)
                val_act_price.append(valid_truth)
                val_adj_close_price.append(valid_adj_close)
                
            # End of outer loop                                        
           
        elapsed_secs = time.time()-start_time
        
        # We report stats on the last portion of the training
        stats_len = int(0.5 * len(train_losses))
            
        metrics_dict = compute_metrics(np.array(val_pred_rt[-stats_len:]),
                                   np.array(val_act_rt[-stats_len:]),
                                   np.array(val_pred_price[-stats_len:]),
                                   np.array(val_act_price[-stats_len:]))

        # This is the results we'll pass back up the stack for the entire training
        result_dict = {'trStats_N':stats_len, 'AvTrLoss':np.mean(train_losses[-stats_len:]),
                   'trAvTrMSE':np.mean(train_mses[-stats_len:]),
                   'trAvValMSE':np.mean(val_mses[-stats_len:]),
                   'trMdnTrMSE':np.median(train_mses[-stats_len:]),
                   'trMdnValMSE':np.median(val_mses[-stats_len:]),
                   'train_mses':train_mses, 'valid_mses':val_mses, 'val_pred_rt':val_pred_rt, 'val_act_rt':val_act_rt, 
                   'val_pred_price':val_pred_price, 'val_act_price':val_act_price, 
                   'val_adj_close_price': val_adj_close_price,
                    'trMDA': metrics_dict['MDA'],
                    'trMAPE': metrics_dict['MAPE'],
                    'trMAE': metrics_dict['MAE'],
                    'trPXCORR': metrics_dict['PXCORR'],
                    'trRXCORR': metrics_dict['RXCORR'],
                    'epoch_train_time':time.time()-epoch_start_time,
                   'train_time':elapsed_secs}
        
#        result_dict.update(metrics_dict)
        
        # Output summary stats at end of outer loop training  
        if (verbosity == 1):
            print('{:4}'.format(i),'{:4.2e}'.format(result_dict['AvTrLoss']),
                          ' {:1.2e}'.format(result_dict['trAvTrMSE']),
                          ' {:4.2e}'.format(result_dict['trAvValMSE']), 
                          ' {:1.2e}'.format(result_dict['trMdnTrMSE']),
                          ' {:4.2e}'.format(result_dict['trMdnValMSE']),                  
                          '  {:2.2f}'.format(result_dict['trMDA']), 
                          '    {:2.2f}'.format(result_dict['trMAPE']), 
                          '    {:2.2e}'.format(result_dict['trMAE']),
                          '{: 1.2f}'.format(result_dict['trPXCORR']), 
                          '  {: 1.2f}'.format(result_dict['trRXCORR']),      
                          '  {:2.2f}'.format(result_dict['epoch_train_time']),                  
                          '  {:3.1f}'.format(result_dict['train_time']/60.0))
            
        with open(basefn + 'train.csv', 'a', newline='') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerow([i, result_dict['AvTrLoss'],
                        result_dict['trAvTrMSE'],
                        result_dict['trAvValMSE'],
                        result_dict['trMdnTrMSE'],
                        result_dict['trMdnValMSE'],
                        result_dict['trMDA'],
                        result_dict['trMAPE'],
                        result_dict['trMAE'],
                        result_dict['trPXCORR'],
                        result_dict['trRXCORR'],
                        result_dict['epoch_train_time'],
                        result_dict['train_time']/60.0,
                        ])
        
        # Periodically perform a trading validation 
        if i%trading_validation_period == 0:
            D,Q,E,A,C = generate_initial_allocation_policy(result_dict, percentiles=percentiles, scale=scale, offset=offset, basefn=basefn, verbosity=0)
            
            # Save both the model and the initial allocation policy so we can recreate for validation / test after training
            # (pick the best model out of a complete training run after analysing the data)
            
            # Make a results directory. If the 'directory' doesn't exist, create it
            if os.path.isdir(basefn + '/saved_models') == False:
                os.mkdir(basefn + '/saved_models')
            
            torch.save(model,basefn + '/saved_models/epoch' + str(i) + '.mdl')
            
            with open(basefn + '/saved_models/DQEAC_dict' + str(i) + '.pkl','wb') as fp:
                pickle.dump({'D':D, 'Q':Q, 'E':E, 'A':A, 'C':C}, fp, protocol=pickle.HIGHEST_PROTOCOL)            

            # Note - prior to version 15, I wasn't deep copying the D,Q,E,A which means they would have been being updated
            # by trading validation.
            trial_results = []
            for trial in range(10):
                # For each trial make a deep copy of things that get modified by the validation process. Make sure that 
                # the model being trained and the allocation policy components are not being modified and we always start
                # validation from the same starting point.
                validation_model = copy.deepcopy(model)
                copy_D = copy.deepcopy(D)
                copy_Q = copy.deepcopy(Q)
                copy_E = copy.deepcopy(E)
                copy_A = copy.deepcopy(A)
                
                validation_model, copy_D, copy_Q, copy_E, copy_A, journal, test_result_dict = test_trading_system(validation_model, valid_seq, valid_labels, copy_D, copy_Q, copy_E, copy_A, percentiles, basefn=basefn+'/saved_models/'+str(i)+'_'+str(trial), scale=scale, offset=offset, iter_per_seq=iter_per_seq, lr = 0.001, dr = 1, verbosity=0)
                
                trial_results.append(test_result_dict)                

            
            mean_trial_MDA = np.mean([v['MDA'] for v in trial_results])
            mean_trial_MSE = np.mean([v['MSE'] for v in trial_results])
            mean_trial_MAE = np.mean([v['MAE'] for v in trial_results])
            mean_trial_MAPE = np.mean([v['MAPE'] for v in trial_results])
            mean_trial_RXCORR = np.mean([v['RXCORR'] for v in trial_results])
            mean_trial_num_trades = np.mean([v['num_trades'] for v in trial_results])
            trial_returns = [v['trade_return'] for v in trial_results]
            mean_trial_trade_return = np.mean(trial_returns)
            median_trial_trade_return = np.median(trial_returns)
            min_trial_trade_return = np.min(trial_returns)
            max_trial_trade_return = np.max(trial_returns)
            range_trial_trade_return = max_trial_trade_return - min_trial_trade_return
            std_trial_trade_return = np.std(trial_returns)
            
            print('Trading Validation - ',
                'MDA:{:2.2f}'.format(mean_trial_MDA),
                '  Returns Correl:{:2.2f}'.format(mean_trial_RXCORR),
                '  Num Trades:{:4}'.format(mean_trial_num_trades),
                '  Return%:{:2.2f}'.format(mean_trial_trade_return))
            
            with open(basefn + 'trade_validation.csv', 'a', newline='') as myfile:
                wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
                wr.writerow([i,mean_trial_MDA, mean_trial_MSE,
                            mean_trial_MAE, mean_trial_MAPE,
                            mean_trial_RXCORR, mean_trial_num_trades, 
                            mean_trial_trade_return, median_trial_trade_return,
                            min_trial_trade_return, max_trial_trade_return,
                            range_trial_trade_return, std_trial_trade_return])

    if plot>=1:
        plot_gradients(model, ave_grads, max_grads, basefn=basefn, display_plot=plot, save_plot=1)
        plotly_gradients(model, min_grads, max_grads, ave_grads, basefn=basefn, display_plot=plot, save_plot=1)
        plot_hidden(h, basefn=basefn, display_plot=plot, save_plot=1)
        plot_lstm_output(lstm_out, basefn=basefn, display_plot=plot, save_plot=1)
        plot_weights(model, basefn=basefn, display_plot=plot, save_plot=1)
        plot_losses(train_mses, val_mses, title='Model Training: Training Loss vs Validation Loss', basefn=basefn, display_plot=plot, save_plot=1)
        plot_returns_hist(val_pred_rt, val_act_rt, title='Model Training: Predicted vs Actual Returns', basefn=basefn+'both_', display_plot=plot, save_plot=1)
        plot_returns_hist(val_pred_rt, val_pred_rt, title='Model Training: Predicted Returns', basefn=basefn+'pred_', display_plot=plot, save_plot=1)
        plot_returns_hist(val_act_rt, val_act_rt, title='Model Training: Actual Returns', basefn=basefn+'act_', display_plot=plot, save_plot=1)    
            
    return model, result_dict



In [ ]:
test_results = []
val_results = []
for i in range(10):
    print("\nTrial:", i)
    print("********\n")
    model, journal, result = run_hypers(verbosity = 1, plot = 1)
    test_results.append(result['test_return_pct'])
    val_results.append(result['val_return_pct'])
    
print(np.mean(np.array(val_results)))
print(np.std(np.array(val_results)))

print(np.mean(np.array(test_results)))
print(np.std(np.array(test_results)))

In [ ]:
torch.set_printoptions(profile="short")

for name in model.named_parameters():
    print(name[0])
    print(name[1][:])

print(model.lstm.weight_ih_l0.size())
print(model.lstm.bias_ih_l0.size())
print(model.lstm.weight_hh_l0.size())
print(model.lstm.bias_hh_l0.size())


print(model.lstm.weight_hh_l1.size())
print(model.lstm.weight_ih_l1.size())

print(model.lstm.weight_hh_l2.size())
print(model.lstm.weight_ih_l2.size())

print(model.linear.weight.size())
print(model.linear.bias.size())





In [ ]:
# Use first asset in list of assets as the source
assets = ['SPY', 'DIA', 'ONEQ', 'IWM']

print(asset_list)
syn_df = asset_dict[asset_list[0]].copy()

display(syn_df.head(5))
print(syn_df.describe())    

asset_dict['SYN'] = syn_df 

trend_list = [(1, 20)]
make_synthetic_df(asset_dict['SYN'], trend_list, 0, 0, 1.4)

display(syn_df.head(5))
print(syn_df.describe())    


In [ ]:
import matplotlib.pyplot as plt
import cv2

# norm_img=cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

# plt.imshow(img, cmap='gray')
fig, ax = plt.subplots(1,6,figsize=(20,12))
ax[0].imshow(model.lstm.weight_ih_l0.cpu().detach().numpy(), cmap='inferno')
ax[1].imshow(model.lstm.weight_hh_l0.cpu().detach().numpy(), cmap='inferno')
ax[2].imshow(model.lstm.weight_ih_l1.cpu().detach().numpy(), cmap='inferno')
ax[3].imshow(model.lstm.weight_hh_l1.cpu().detach().numpy(), cmap='inferno')
ax[4].imshow(model.lstm.weight_ih_l2.cpu().detach().numpy(), cmap='inferno')
ax[5].imshow(model.lstm.weight_hh_l2.cpu().detach().numpy(), cmap='inferno')

# ih_l0 - has dimension 6 x 512

# fig.tight_layout()

In [ ]:
# Stock market indices
# assets = ['^GSPC', '^DJI', '^IXIC', '^RUT']

assets = ['SPY', 'DIA', 'ONEQ', 'IWM']

asset_list = assets

In [ ]:
# Iterate through index list, check if we already have the data (load it) or go get it from Yahoo! (save it)
# Make a dictionary of dataframes containing all the data

start_date = '01-01-2005'
# stop_date = '05-01-2018' # First version of paper
stop_date = '12-20-2019' # Final version of paper
directory = '../../../data/yahoo_data'

# If the 'directory' doesn't exist, create it
#if os.path.isdir(directory) == False:
#    os.mkdir(directory)

asset_dict = {}
for asset in asset_list:
    df = get_data(asset_name=asset, directory=directory, start_date=start_date, stop_date=stop_date)    
    asset_dict[asset] = df 

In [ ]:
for asset in asset_list:
    display(asset_dict[asset].head(5))
    print(asset, asset_dict[asset].describe())

In [ ]:
# Plot all the assets in Plotly interactive charts with a Candlestick chart
for asset in asset_list:
    plotly_candlestick(asset_dict, asset)

In [ ]:
# Before we split the data into train, validation and test sets let's add the Prev Adj Close feature and
# remove the Date and Volume columns (which are not used)

for asset in asset_list:
    asset_dict[asset] = add_features_to_df(asset_dict[asset])       
    display(asset, asset_dict[asset].head(5))

In [ ]:
# Define the start and end dates for each of the sets - start and end dates are inclusive

# Training set
train_start = '01-01-2005'
train_end = '01-01-2008'

# Validation set
valid_start = '01-02-2008'
valid_end = '12-31-2009'

# A combined train / valid set
train_val_start = '01-01-2005'
train_val_end = '12-31-2009'

# Pre-test set (120 days prior to test period, to generate allocation policy for testing)
pre_test_start = '07-15-2009'
pre_test_end = '12-31-2009'

# Out of sample test set
test_start = '01-01-2010'
test_end = '12-20-2019'


In [ ]:
# paper_baseline = {'SPY':136.4, 'DIA':136.6, 'ONEQ':228.9, 'IWM':163.5} # Original paper
paper_baseline = {'SPY':188.87, 'DIA':172.87, 'ONEQ':293.32, 'IWM':167.34, 'SYN':0} # Latest paper - table 12


train_asset_dict = {}
valid_asset_dict = {}
test_asset_dict = {}
train_val_asset_dict = {}
pre_test_asset_dict = {}

for asset in asset_list:
    # Make a dataframe for this asset
    df = asset_dict[asset]
    
    # Mask off the training data and assign result to a train_asset dictionary
    mask = (df['Date'] >= train_start) & (df['Date'] <= train_end)
    train_asset_dict[asset] = df.loc[mask]
    
    # Mask off the validation data and assign result to a valid_asset dictionary
    mask = (df['Date'] >= valid_start) & (df['Date'] <= valid_end)
    valid_asset_dict[asset] = df.loc[mask]
    
    # Mask off the test data and assign result to a test_asset dictionary
    mask = (df['Date'] >= test_start) & (df['Date'] <= test_end)
    test_asset_dict[asset] = df.loc[mask]
    
    # Mask off the combined train / validation data and assign result to a train_val_asset dictionary
    mask = (df['Date'] >= train_val_start) & (df['Date'] <= train_val_end)
    train_val_asset_dict[asset] = df.loc[mask]

    # Mask off the pre_test data and assign result to a pre_test_asset dictionary
    mask = (df['Date'] >= pre_test_start) & (df['Date'] <= pre_test_end)
    pre_test_asset_dict[asset] = df.loc[mask]
    
    # Reset the indices to start from 0 again for each of the dataframes
    train_asset_dict[asset].index = np.arange(train_asset_dict[asset].shape[0])    
    valid_asset_dict[asset].index = np.arange(valid_asset_dict[asset].shape[0])    
    test_asset_dict[asset].index = np.arange(test_asset_dict[asset].shape[0])
    train_val_asset_dict[asset].index = np.arange(train_val_asset_dict[asset].shape[0])    
    pre_test_asset_dict[asset].index = np.arange(pre_test_asset_dict[asset].shape[0])    

    
#    display(test_asset_dict[asset])

print("Adjusted Close to Adjusted Close:")
print('Asset\tStart Date\t\tBeg$\tEnd Date\t\tEnd$\tGain%\tPaper%')
for asset in asset_list:
    # Print the buy and hold returns over the entire test period (the papers baseline)
    start = test_asset_dict[asset]['Adj Close'].loc[0]
    end = test_asset_dict[asset]['Adj Close'].loc[test_asset_dict[asset].shape[0]-1]
    gain = 100.0 * (end - start) / start
    
    print(asset, '\t', test_asset_dict[asset]['Date'].loc[0], '\t{:1.2f}'.format(start), '\t', 
          test_asset_dict[asset]['Date'].loc[test_asset_dict[asset].shape[0]-1], 
          '\t{:1.2f}'.format(end), '\t{:1.2f}'.format(gain), '\t{:1.2f}'.format(paper_baseline[asset]))

print("\nClose to Close:")
print('Asset\tStart Date\t\tBeg$\tEnd Date\t\tEnd$\tGain%\tPaper%')
for asset in asset_list:
    # Print the buy and hold returns over the entire test period (the papers baseline)
    start = test_asset_dict[asset]['Close'].loc[0]
    end = test_asset_dict[asset]['Close'].loc[test_asset_dict[asset].shape[0]-1]
    gain = 100.0 * (end - start) / start
    
    print(asset, '\t', test_asset_dict[asset]['Date'].loc[0], '\t{:1.2f}'.format(start), '\t', 
          test_asset_dict[asset]['Date'].loc[test_asset_dict[asset].shape[0]-1], 
          '\t{:1.2f}'.format(end), '\t{:1.2f}'.format(gain), '\t{:1.2f}'.format(paper_baseline[asset]))
    
print("\nOpen to Close:")
print('Asset\tStart Date\t\tBeg$\tEnd Date\t\tEnd$\tGain%\tPaper%')
for asset in asset_list:
    # Print the buy and hold returns over the entire test period (the papers baseline)
    start = test_asset_dict[asset]['Open'].loc[0]
    end = test_asset_dict[asset]['Close'].loc[test_asset_dict[asset].shape[0]-1]
    gain = 100.0 * (end - start) / start
    
    print(asset, '\t', test_asset_dict[asset]['Date'].loc[0], '\t{:1.2f}'.format(start), '\t', 
          test_asset_dict[asset]['Date'].loc[test_asset_dict[asset].shape[0]-1], 
          '\t{:1.2f}'.format(end), '\t{:1.2f}'.format(gain), '\t{:1.2f}'.format(paper_baseline[asset]))    
    
# It looks like the paper used Close-Close or Open-Close rather than Adjusted Close to compute gains over the test period. 
# They are a lot closer to the paper quoted returns and better correlated
    

In [ ]:
print("Train set length:", len(train_asset_dict['SPY']))
print("Valid set length", len(valid_asset_dict['SPY']))
print("Train / Valid set length", len(train_val_asset_dict['SPY']))
print("Pre-test set", len(pre_test_asset_dict['SPY']))
print("Test set",len(test_asset_dict['SPY']))